# Combine webnlg-gpt-explanations and text_to_triplets Datasets

In [1]:
from datasets import load_dataset, DatasetDict, Dataset, disable_caching
import pandas as pd
import numpy as np
import csv
import re

In [2]:
disable_caching()
text_to_triples = load_dataset('UofA-LINGO/text_to_triplets', download_mode='force_redownload')
webnlg_clean = load_dataset('UofA-LINGO/webnlg-cleaned-noins', download_mode='force_redownload')
webnlg = load_dataset('UofA-LINGO/webnlg-gpt-explanations', download_mode='force_redownload')
ske = load_dataset('UofA-LINGO/ske2019', download_mode='force_redownload')
fewrel = load_dataset('UofA-LINGO/FewRel-train-noins', download_mode='force_redownload')
docred = load_dataset('UofA-LINGO/DocRED-train-noins', download_mode='force_redownload')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2155 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--text_to_triplets-e363ff5423825b5c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3053 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--DocRED-train-noins-2ac855fd131f3594/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
print(text_to_triples)
print(text_to_triples["test"]["context"][50])
print(text_to_triples["test"]["response"][50])
print(webnlg)
print(ske)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response'],
        num_rows: 13211
    })
    test: Dataset({
        features: ['instruction', 'context', 'response'],
        num_rows: 2155
    })
})
McVeagh of the South Seas is a film directed by Harry Carey and Cyril Bruce. Carey was born in 1878 and he was the lead actor of the film. This movie can be found on the IMDb by inputting the Id of 0004319.
['McVeagh_of_the_South_Seas | imdbId | 0004319', 'McVeagh_of_the_South_Seas | director | Cyril_Bruce', 'McVeagh_of_the_South_Seas | director | Harry_Carey_(actor_born_1878)', 'McVeagh_of_the_South_Seas | starring | Harry_Carey_(actor_born_1878)']
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 13211
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 194747
    })
    test: Dataset({
        features: ['instruction', 'input', 'out

In [3]:
ttt_pd = pd.DataFrame(text_to_triples['train'])
ttt_pd = ttt_pd.rename(columns={"instruction":"instruction", "context":"input", "response":"original_output"})
web_pd = pd.DataFrame(webnlg['train'])
web_pd = web_pd.rename(columns={"output":"original_output"})
web_cl = pd.DataFrame(webnlg_clean['train'])
web_cl = web_cl.rename(columns={"output":"original_output"})
ske_pd = pd.DataFrame(ske['train'])
ske_pd = ske_pd.rename(columns={"output":"original_output"})
ske_val = pd.DataFrame(ske['valid'])
ske_val = ske_val.rename(columns={"output":"original_output"})
few_pd = pd.DataFrame(fewrel['train'])
doc_pd = pd.DataFrame(docred['train'])

In [4]:
doc_pd

,input,output
0,"Zest Airways, Inc. operated as AirAsia Zest (f...",(Asian Spirit | country | Republic of the Phil...
1,The short-beaked common dolphin (Delphinus del...,
2,Niklas Bergqvist (born 6 October 1962 in Stock...,(Niklas Bergqvist | date of birth | 6 October ...
3,Pacific Fair is a major shopping centre in Bro...,(Coles | country | Australia)\n(Toys ' R ' Us ...
4,The Ministry of Home Security was a British go...,(Home Office | country | British)\n(Herbert Mo...
...,...,...
3048,The United African National Council (UANC) was...,(ZANU - PF | country | Zimbabwean)\n(UANC Part...
3049,BBC Sessions is a live album by the British ro...,(I Feel Free – Ultimate Cream | performer | Cr...
3050,The Atheist Centre is an institution founded b...,(Saraswathi Gora | religion or worldview | Ath...
3051,The Battle of Naseby was a decisive engagement...,(Thomas Fairfax | military branch | Parliament...


In [7]:
few_pd

,input,output
0,Merpati flight 106 departed Jakarta (CGK) on a...,(tjq | place served by transport hub | tanjung...
1,The name was at one point changed to Nottingha...,(east midlands airport | place served by trans...
2,It is a four-level stack interchange near Fort...,(fort lauderdale-hollywood international airpo...
3,It is the main alternate of Jinnah Internation...,(jinnah international airport | place served b...
4,Nearby airports include Akwa Ibom Airport at O...,(margaret ekpo international airport | place s...
...,...,...
44795,"He won the first prize for piano, teaching, ch...",(rimsky-korsakov conservatory | located in the...
44796,Tydavnet / Monaghan and Geel were officially t...,(tydavnet | located in the administrative terr...
44797,Serbo-Croatian is a lingua franca in several o...,(serbia | located in the administrative territ...
44798,A pivotal moment in UCSF history was the deal ...,(mission bay | located in the administrative t...


In [18]:
web_pd

,instruction,input,original_output
0,Transform the subsequent text into semantic tr...,"The Aarhus is the airport of Aarhus, Denmark.","To extract triplets from the given text, we ne..."
1,Interpret the subsequent text as semantic trip...,Aarhus airport serves the city of Aarhus.,"To extract triplets from the given text, you c..."
2,Bring out triplets from the text that's below.,Aarhus Airport is 25 metres above sea level.,"To extract triplets from the given text, we ca..."
3,Transfigure the text that follows into semanti...,Aarhus Airport is located in Tirstrup.,"To extract triplets from the given text, you c..."
4,Decipher the text underneath into triplets.,Aarhus Airport is operated by Aarhus Lufthavn ...,"To extract triplets from the given text, we ca..."
...,...,...,...
13206,Transfigure the text that follows into semanti...,In 1928 the School of Business and Social Scie...,"To extract triplets from the given text, we ca..."
13207,Convert the following text into semantic tripl...,The Dean of the School of Business and Social ...,"To extract triplets from the given text, we ne..."
13208,Convert the following text into semantic tripl...,The city of Aarhus in Denmark is the location ...,"To extract triplets from the given text, we ca..."
13209,Reformat the following text into semantic trip...,The campus address of the Acharya Institute of...,"To extract triplets from the given text, we ne..."


In [19]:
web_cl

,input,original_output
0,"The Aarhus is the airport of Aarhus, Denmark.","['Aarhus_Airport | cityServed | Aarhus,_Denmark']"
1,Aarhus airport serves the city of Aarhus.,['Aarhus_Airport | cityServed | Aarhus']
2,Aarhus Airport is 25 metres above sea level.,['Aarhus_Airport | elevation | 25.0']
3,Aarhus Airport is located in Tirstrup.,['Aarhus_Airport | location | Tirstrup']
4,Aarhus Airport is operated by Aarhus Lufthavn ...,"['Aarhus_Airport | operator | ""Aarhus Lufthavn..."
...,...,...
13206,In 1928 the School of Business and Social Scie...,"['Aarhus_University,_School_of_Business_and_So..."
13207,The Dean of the School of Business and Social ...,"['Aarhus_University,_School_of_Business_and_So..."
13208,The city of Aarhus in Denmark is the location ...,"['Aarhus_University,_School_of_Business_and_So..."
13209,The campus address of the Acharya Institute of...,['Visvesvaraya_Technological_University | city...


In [20]:
ske_pd

,instruction,input,original_output
0,extracting triples from following text,如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈,['喜剧之王 | 主演 | 周星驰']
1,extracting triples from following text,茶树茶网蝽，Stephanitis chinensis Drake，属半翅目网蝽科冠网椿属的...,['茶树茶网蝽 | 目 | 半翅目']
2,extracting triples from following text,丝角蝗科，Oedipodidae，昆虫纲直翅目蝗总科的一个科,['丝角蝗科 | 目 | 直翅目']
3,extracting triples from following text,爱德华·尼科·埃尔南迪斯（1986-），是一位身高只有70公分哥伦比亚男子，体重10公斤，只...,"['爱德华·尼科·埃尔南迪斯 | 身高 | 70公分', '爱德华·尼科·埃尔南迪斯 | 出..."
4,extracting triples from following text,《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-...,"['逐风行 | 连载网站 | 纵横中文网', '逐风行 | 作者 | 清水秋风']"
...,...,...,...
194742,extracting triples from following text,7气候云林县属南亚热带湿润气候,['云林县 | 气候 | 南亚热带湿润气候']
194743,extracting triples from following text,《可持续建筑设计实践》是2006年中国建筑工业出版社出版的图书，作者是斯泰里奥斯·普莱尼奥斯，纪雁,"['可持续建筑设计实践 | 作者 | 斯泰里奥斯·普莱尼奥斯', '可持续建筑设计实践 | ..."
194744,extracting triples from following text,武汉土木石建筑装饰设计工程有限公司于2012年10月25日成立,['武汉土木石建筑装饰设计工程有限公司 | 成立日期 | 2012年10月25日']
194745,extracting triples from following text,《网游之最强时代》是创世中文网连载的小说，作者是江山,"['网游之最强时代 | 连载网站 | 创世中文网', '网游之最强时代 | 作者 | 江山']"


In [6]:
print(web_pd['original_output'][0])

To extract triplets from the given text, we need to identify the subject, predicate, and object. 

Subject: "Aarhus Airport"
Predicate: "cityServed"
Object: "Aarhus, Denmark"

The property "cityServed" is derived from the context of the sentence, where it implies that the airport serves the city of Aarhus.

Therefore, here is the answer in the correct format:

['Aarhus_Airport | cityServed | "Aarhus, Denmark"']


## Convert Triplets to (S | P | O) Format

In [ ]:
('Aarhus_Airport | cityServed | "Aarhus, Denmark"')\n('Aarhus | country | Denmark')

In [5]:
order = ["instruction", "input", "output", "original_output"]

In [6]:
triples = []
cnt = 0
l = len("Therefore, here is the answer in the correct format:\n\n")
for out in ttt_pd['original_output']:
    new_triples = ""
    out = re.split("\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split('|')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [7]:
triples = pd.DataFrame(triples, columns=['output'])
ttt_pd.loc[:, "output"] = ttt_pd.index.map(triples['output'])
ttt_pd = ttt_pd.reindex(columns=order)
ttt_pd

,instruction,input,output,original_output
0,Generating triplets from the given text:,"The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Deriving triplets by analyzing the provided text:,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,Obtaining triples from the given text:,Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Deriving triplets by analyzing the provided text:,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Extracting triples by processing the given text:,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
...,...,...,...,...
13206,Creating triples by extracting information fro...,In 1928 the School of Business and Social Scie...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13207,Constructing triples by analyzing the given text:,The Dean of the School of Business and Social ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13208,Creating triples by extracting information fro...,The city of Aarhus in Denmark is the location ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13209,Generating triplets by parsing the specified t...,The campus address of the Acharya Institute of...,(Visvesvaraya_Technological_University | city ...,['Visvesvaraya_Technological_University | city...


In [13]:
triples = []
cnt = 0
l = len("Therefore, here is the answer in the correct format:\n\n")
for out in web_pd['original_output']:
    new_triples = out[:out.find("Therefore, here is the answer in the correct format:\n\n")+l]
    out = out[out.find("Therefore, here is the answer in the correct format:\n\n")+l:]
    out = re.split("\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split('|')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [14]:
triples = pd.DataFrame(triples, columns=['output'])
web_pd.loc[:, "output"] = web_pd.index.map(triples['output'])
web_pd = web_pd.reindex(columns=order)
web_pd

,instruction,input,output,original_output
0,Transform the subsequent text into semantic tr...,"The Aarhus is the airport of Aarhus, Denmark.","To extract triplets from the given text, we ne...","To extract triplets from the given text, we ne..."
1,Interpret the subsequent text as semantic trip...,Aarhus airport serves the city of Aarhus.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
2,Bring out triplets from the text that's below.,Aarhus Airport is 25 metres above sea level.,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
3,Transfigure the text that follows into semanti...,Aarhus Airport is located in Tirstrup.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
4,Decipher the text underneath into triplets.,Aarhus Airport is operated by Aarhus Lufthavn ...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
...,...,...,...,...
13206,Transfigure the text that follows into semanti...,In 1928 the School of Business and Social Scie...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
13207,Convert the following text into semantic tripl...,The Dean of the School of Business and Social ...,"To extract triplets from the given text, we ne...","To extract triplets from the given text, we ne..."
13208,Convert the following text into semantic tripl...,The city of Aarhus in Denmark is the location ...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
13209,Reformat the following text into semantic trip...,The campus address of the Acharya Institute of...,"To extract triplets from the given text, we ne...","To extract triplets from the given text, we ne..."


In [26]:
print(web_pd['output'][0])

To extract triplets from the given text, we need to identify the subject, predicate, and object. 

Subject: "Aarhus Airport"
Predicate: "cityServed"
Object: "Aarhus, Denmark"

The property "cityServed" is derived from the context of the sentence, where it implies that the airport serves the city of Aarhus.

Therefore, here is the answer in the correct format:

(Aarhus_Airport | cityServed | "Aarhus, Denmark")


In [16]:
web_cl['original_output'][1803]

"['Asam_pedas | region | Indonesia', 'Asam_pedas | region | Malaysia']"

In [17]:
triples = []
cnt = 0
l = len("Therefore, here is the answer in the correct format:\n\n")
for out in web_cl['original_output']:
    new_triples = ""
    out = re.split("\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split('|')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [18]:
triples = pd.DataFrame(triples, columns=['output'])
web_cl.loc[:, "output"] = web_cl.index.map(triples['output'])
web_cl = web_cl.reindex(columns=order)
web_cl

,instruction,input,output,original_output
0,NaN,"The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | Aarhus,_Denmark)","['Aarhus_Airport | cityServed | Aarhus,_Denmark']"
1,NaN,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,NaN,Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevation | 25.0),['Aarhus_Airport | elevation | 25.0']
3,NaN,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,NaN,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operator | ""Aarhus Lufthavn ...","['Aarhus_Airport | operator | ""Aarhus Lufthavn..."
...,...,...,...,...
13206,NaN,In 1928 the School of Business and Social Scie...,"(Aarhus_University,_School_of_Business_and_Soc...","['Aarhus_University,_School_of_Business_and_So..."
13207,NaN,The Dean of the School of Business and Social ...,"(Aarhus_University,_School_of_Business_and_Soc...","['Aarhus_University,_School_of_Business_and_So..."
13208,NaN,The city of Aarhus in Denmark is the location ...,"(Aarhus_University,_School_of_Business_and_Soc...","['Aarhus_University,_School_of_Business_and_So..."
13209,NaN,The campus address of the Acharya Institute of...,(Visvesvaraya_Technological_University | city ...,['Visvesvaraya_Technological_University | city...


In [27]:
triples = []
cnt = 0
l = len("Therefore, here is the answer in the correct format:\n\n")
for out in ske_pd['original_output']:
    new_triples = ""
    if out == '[]':
        print(cnt)
        triples.append(None)
        continue
    out = re.split("\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split(' | ')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

23237
24476
30459
34652
45818
60831
62109
64306
95033
102874
106704
109662
111127
117974
140549
143337
145468
166974
172287


In [28]:
print(ske_pd['input'][23237])

基本信息曼 ( 1871～1950 ) Mann，Heinrich 德国作家


In [29]:
triples = pd.DataFrame(triples, columns=['output'])
ske_pd.loc[:, "output"] = ske_pd.index.map(triples['output'])
ske_pd = ske_pd.reindex(columns=order)
ske_pd

,instruction,input,output,original_output
0,extracting triples from following text,如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈,(喜剧之王 | 主演 | 周星驰),['喜剧之王 | 主演 | 周星驰']
1,extracting triples from following text,茶树茶网蝽，Stephanitis chinensis Drake，属半翅目网蝽科冠网椿属的...,(茶树茶网蝽 | 目 | 半翅目),['茶树茶网蝽 | 目 | 半翅目']
2,extracting triples from following text,丝角蝗科，Oedipodidae，昆虫纲直翅目蝗总科的一个科,(丝角蝗科 | 目 | 直翅目),['丝角蝗科 | 目 | 直翅目']
3,extracting triples from following text,爱德华·尼科·埃尔南迪斯（1986-），是一位身高只有70公分哥伦比亚男子，体重10公斤，只...,(爱德华·尼科·埃尔南迪斯 | 身高 | 70公分)\n(爱德华·尼科·埃尔南迪斯 | 出生...,"['爱德华·尼科·埃尔南迪斯 | 身高 | 70公分', '爱德华·尼科·埃尔南迪斯 | 出..."
4,extracting triples from following text,《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-...,(逐风行 | 连载网站 | 纵横中文网)\n(逐风行 | 作者 | 清水秋风),"['逐风行 | 连载网站 | 纵横中文网', '逐风行 | 作者 | 清水秋风']"
...,...,...,...,...
194742,extracting triples from following text,7气候云林县属南亚热带湿润气候,(云林县 | 气候 | 南亚热带湿润气候),['云林县 | 气候 | 南亚热带湿润气候']
194743,extracting triples from following text,《可持续建筑设计实践》是2006年中国建筑工业出版社出版的图书，作者是斯泰里奥斯·普莱尼奥斯，纪雁,(可持续建筑设计实践 | 作者 | 斯泰里奥斯·普莱尼奥斯)\n(可持续建筑设计实践 | 出...,"['可持续建筑设计实践 | 作者 | 斯泰里奥斯·普莱尼奥斯', '可持续建筑设计实践 | ..."
194744,extracting triples from following text,武汉土木石建筑装饰设计工程有限公司于2012年10月25日成立,(武汉土木石建筑装饰设计工程有限公司 | 成立日期 | 2012年10月25日),['武汉土木石建筑装饰设计工程有限公司 | 成立日期 | 2012年10月25日']
194745,extracting triples from following text,《网游之最强时代》是创世中文网连载的小说，作者是江山,(网游之最强时代 | 连载网站 | 创世中文网)\n(网游之最强时代 | 作者 | 江山),"['网游之最强时代 | 连载网站 | 创世中文网', '网游之最强时代 | 作者 | 江山']"


In [30]:
ske_pd[ske_pd.isna().any(axis=1)]

,instruction,input,output,original_output
23237,extracting triples from following text,基本信息曼 ( 1871～1950 ) Mann，Heinrich 德国作家,None,[]
24477,extracting triples from following text,赵坤，男，汉族，1970年4月出生，固原市西吉县人，在职大学学历，1994年1月加入中国共产...,None,[]
30461,extracting triples from following text,兰胜强，男，汉族，1957年12月出生，乳山市乳山口镇人，1982年7月参加工作，1976年...,None,[]
34655,extracting triples from following text,于波，男，汉族，1966年5月出生，中共党员，大学文化，学士学位，现任哈尔滨市委办公厅副局级干部，,None,[]
45822,extracting triples from following text,刘卫光，男，汉族，山东省莱阳市人，中共党员，大学文化,None,[]
60836,extracting triples from following text,邹诗机，男，汉族，1973年4月生，贵州省普安县人，中央党校大学文化，1992年8月参加工作...,None,[]
62115,extracting triples from following text,雷斌，男，汉族，1970年1月出生，桂阳县东城乡人，中共党员，大学文化,None,[]
64313,extracting triples from following text,贺文萍，女，汉族，1953年8月出生，四川广元人，大学文化,None,[]
95041,extracting triples from following text,"薛喜枚，女，汉族，1977年出生，山东沂南人 , 2005年7月毕业于中科院成都生物研究所，...",None,[]
102883,extracting triples from following text,任埃成，副主任:男，汉族，达旗人，1965年4月出生，中共党员，大学文化,None,[]


In [32]:
ske_pd.dropna(inplace=True)
ske_pd.reset_index(drop=True, inplace=True)

In [10]:
triples = []
cnt = 0
l = len("Therefore, here is the answer in the correct format:\n\n")
for out in ske_val['original_output']:
    new_triples = ""
    if out == '[]':
        print(cnt)
        triples.append(None)
        continue
    out = re.split("\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split(' | ')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [11]:
triples = pd.DataFrame(triples, columns=['output'])
ske_val.loc[:, "output"] = ske_val.index.map(triples['output'])
ske_val = ske_val.reindex(columns=order)
ske_val

,instruction,input,output,original_output
0,extracting triples from following text,查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥...,(查尔斯·阿兰基斯 | 出生地 | 圣地亚哥)\n(查尔斯·阿兰基斯 | 出生日期 | 19...,"['查尔斯·阿兰基斯 | 出生地 | 圣地亚哥', '查尔斯·阿兰基斯 | 出生日期 | 1..."
1,extracting triples from following text,《离开》是由张宇谱曲，演唱,(离开 | 歌手 | 张宇)\n(离开 | 作曲 | 张宇),"['离开 | 歌手 | 张宇', '离开 | 作曲 | 张宇']"
2,extracting triples from following text,《愤怒的唐僧》由北京吴意波影视文化工作室与优酷电视剧频道联合制作，故事以喜剧元素为主，讲述唐...,(愤怒的唐僧 | 出品公司 | 北京吴意波影视文化工作室)\n(愤怒的唐僧 | 导演 | 吴意波),"['愤怒的唐僧 | 出品公司 | 北京吴意波影视文化工作室', '愤怒的唐僧 | 导演 | ..."
3,extracting triples from following text,李治即位后，萧淑妃受宠，王皇后为了排挤萧淑妃，答应李治让身在感业寺的武则天续起头发，重新纳入后宫,(李治 | 妻子 | 萧淑妃)\n(萧淑妃 | 丈夫 | 李治),"['李治 | 妻子 | 萧淑妃', '萧淑妃 | 丈夫 | 李治']"
4,extracting triples from following text,《工业4 . 0》是2015年机械工业出版社出版的图书，作者是（德）阿尔冯斯·波特霍夫，恩斯...,(工业4.0 | 作者 | 恩斯特·安德雷亚斯·哈特曼)\n(工业4.0 | 作者 | 阿尔...,"['工业4.0 | 作者 | 恩斯特·安德雷亚斯·哈特曼', '工业4.0 | 作者 | 阿..."
...,...,...,...,...
21634,extracting triples from following text,7气候云林县属南亚热带湿润气候,(云林县 | 气候 | 南亚热带湿润气候),['云林县 | 气候 | 南亚热带湿润气候']
21635,extracting triples from following text,《可持续建筑设计实践》是2006年中国建筑工业出版社出版的图书，作者是斯泰里奥斯·普莱尼奥斯，纪雁,(可持续建筑设计实践 | 作者 | 斯泰里奥斯·普莱尼奥斯)\n(可持续建筑设计实践 | 出...,"['可持续建筑设计实践 | 作者 | 斯泰里奥斯·普莱尼奥斯', '可持续建筑设计实践 | ..."
21636,extracting triples from following text,武汉土木石建筑装饰设计工程有限公司于2012年10月25日成立,(武汉土木石建筑装饰设计工程有限公司 | 成立日期 | 2012年10月25日),['武汉土木石建筑装饰设计工程有限公司 | 成立日期 | 2012年10月25日']
21637,extracting triples from following text,《网游之最强时代》是创世中文网连载的小说，作者是江山,(网游之最强时代 | 连载网站 | 创世中文网)\n(网游之最强时代 | 作者 | 江山),"['网游之最强时代 | 连载网站 | 创世中文网', '网游之最强时代 | 作者 | 江山']"


In [42]:
ske_val = ske_val.dropna(axis=0)
ske_val[ske_val.isna().any(axis=1)]

,instruction,input,output,original_output


## Randomize new instructions for the data

In [8]:
# instructions_base_short = [
#     "Derive semantic triplets from the subsequent text.",
#     "Retrieve triplets from the following text.",
#     "Pull out triplets from the text underneath.",
#     "Draw out semantic triplets from the text provided below.",
#     "Elicit triplets from the subsequent textual content.",
#     "Generate semantic triplets from the text beneath.",
#     "Obtain triplets from the text that follows.",
#     "Bring out triplets from the text that's below.",
#     "Convert the following text into semantic triplets.",
#     "Turn the text below into triplets.",
#     "Transform the subsequent text into semantic triplets.",
#     "Transpose the text underneath into triplets.",
#     "Reformat the following text into semantic triplets.",
#     "Process the text below into triplets.",
#     "Interpret the subsequent text as semantic triplets.",
#     "Decipher the text underneath into triplets.",
#     "Transcribe the following text into semantic triplets.",
#     "Create triplets from the text below.",
#     "Alter the text that follows into semantic triplets.",
#     "Construct semantic triplets from the text provided below.",
# ]
instructions_base_short = [
    "Convert the following text into semantic triplets.",
    "Imagine you're an expert at deciphering text, capable of pulling out all entities and their relationships. When provided a text, your task is to identify all possible triplets, following the format: (subject | relationship | object).",
    "As someone proficient in text parsing, you should be able to identify all entities and their associated relationships from a given text. I will hand you a text, and your job is to derive all potential triples, using the structure: (subject | relationship | object).",  
    "Envision yourself as skilled in analyzing text to the point of being able to draw out all entities and their correlations. Once I give you a text, your assignment is to generate all the plausible triples, formatted as: (subject | relationship | object).",
    "Assume you're highly competent in scrutinizing a piece of text and successfully distilling all its entities along with their connections. I'll provide a text, and you are to extract every possible triplet, following the convention: (subject | relationship | object).",
    "Picture yourself as adept at understanding and dissecting text, capable of efficiently identifying all entities and their interrelationships. Given a text, it's your task to isolate all potential triples, in the format of: (subject | relationship | object).",
    "Suppose you're an expert in text dissection, able to accurately pull out all entities and their ties from any text. I will provide you with a text, and your task is to excavate all conceivable triples, following the template: (subject | relationship | object).",
    "Pretend you are highly skilled in breaking down a text, with the capability to effectively discern all the entities and their interactions. Once I supply you with a text, your mission is to extract all potential triple structures, in the form of: (subject | relationship | object).",
    "Think of yourself as efficient in deconstructing a text and precisely identifying all the entities and their interrelations. I'll furnish you with a text and your job is to gather all potential triples, adhering to the pattern: (subject | relationship | object).",
    "Suppose you're a maestro in textual analysis, capable of efficiently extracting all entities and their associations. Upon receiving a text from me, it's your duty to produce all possible triples, consistent with the arrangement: (subject | relationship | object).",
    "Pretend that you are great at parsing a text and can effectively extract all entities and their relationships from a text. I will give you a text and you extract all the possible triples.  The output format is: (subject | relationship | object).",
]

In [31]:
# randomly pick one instrcution for each sample
ske_pd.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(ske_pd))
ske_pd.head(50)

,instruction,input,output,original_output
0,Pretend that you are great at parsing a text a...,如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈,(喜剧之王 | 主演 | 周星驰),['喜剧之王 | 主演 | 周星驰']
1,Envision yourself as skilled in analyzing text...,茶树茶网蝽，Stephanitis chinensis Drake，属半翅目网蝽科冠网椿属的...,(茶树茶网蝽 | 目 | 半翅目),['茶树茶网蝽 | 目 | 半翅目']
2,"Imagine you're an expert at deciphering text, ...",丝角蝗科，Oedipodidae，昆虫纲直翅目蝗总科的一个科,(丝角蝗科 | 目 | 直翅目),['丝角蝗科 | 目 | 直翅目']
3,Assume you're highly competent in scrutinizing...,爱德华·尼科·埃尔南迪斯（1986-），是一位身高只有70公分哥伦比亚男子，体重10公斤，只...,(爱德华·尼科·埃尔南迪斯 | 身高 | 70公分)\n(爱德华·尼科·埃尔南迪斯 | 出生...,"['爱德华·尼科·埃尔南迪斯 | 身高 | 70公分', '爱德华·尼科·埃尔南迪斯 | 出..."
4,Envision yourself as skilled in analyzing text...,《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-...,(逐风行 | 连载网站 | 纵横中文网)\n(逐风行 | 作者 | 清水秋风),"['逐风行 | 连载网站 | 纵横中文网', '逐风行 | 作者 | 清水秋风']"
5,Think of yourself as efficient in deconstructi...,禅意歌者刘珂矣《一袖云》中诉知己…绵柔纯净的女声，将心中的万水千山尽意勾勒于这清素画音中,(一袖云 | 歌手 | 刘珂矣),['一袖云 | 歌手 | 刘珂矣']
6,"As someone proficient in text parsing, you sho...",南迦帕尔巴特峰，8125米,(南迦帕尔巴特峰 | 海拔 | 8125米),['南迦帕尔巴特峰 | 海拔 | 8125米']
7,"Suppose you're an expert in text dissection, a...",《身外身梦中梦》是连载于晋江文学城的一部原创类小说，作者是苍生笑,(身外身梦中梦 | 作者 | 苍生笑),['身外身梦中梦 | 作者 | 苍生笑']
8,Picture yourself as adept at understanding and...,谢万松，字树人，湖北省武汉市人，武汉钢铁集团公司联合焦化公司退体职工，生于1940年,(谢万松 | 出生地 | 湖北省武汉市),['谢万松 | 出生地 | 湖北省武汉市']
9,"As someone proficient in text parsing, you sho...",《娘家的故事第二部》是张玲执导，林在培、何赛飞等主演的电视剧,(娘家的故事第二部 | 导演 | 张玲)\n(娘家的故事第二部 | 主演 | 林在培)\n(...,"['娘家的故事第二部 | 导演 | 张玲', '娘家的故事第二部 | 主演 | 林在培', ..."


In [9]:
few_pd.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(few_pd))
few_pd.head(50)

,input,output,instruction
0,Merpati flight 106 departed Jakarta (CGK) on a...,(tjq | place served by transport hub | tanjung...,"As someone proficient in text parsing, you sho..."
1,The name was at one point changed to Nottingha...,(east midlands airport | place served by trans...,Think of yourself as efficient in deconstructi...
2,It is a four-level stack interchange near Fort...,(fort lauderdale-hollywood international airpo...,"As someone proficient in text parsing, you sho..."
3,It is the main alternate of Jinnah Internation...,(jinnah international airport | place served b...,"Imagine you're an expert at deciphering text, ..."
4,Nearby airports include Akwa Ibom Airport at O...,(margaret ekpo international airport | place s...,Think of yourself as efficient in deconstructi...
5,"Pakyong Airport, a Greenfield project, is unde...",(pakyong airport | place served by transport h...,"As someone proficient in text parsing, you sho..."
6,Its hub is Tinson Pen Aerodrome in Kingston (K...,(tinson pen aerodrome | place served by transp...,Think of yourself as efficient in deconstructi...
7,He has postage stamps issued in his honour and...,(noumérat – moufdi zakaria airport | place ser...,Pretend you are highly skilled in breaking dow...
8,By Airway ; \n Nearest airport is Kazi Nazrul ...,(kazi nazrul islam airport | place served by t...,Envision yourself as skilled in analyzing text...
9,The nearest airport is Robin Hood Airport at D...,(robin hood airport | place served by transpor...,Pretend you are highly skilled in breaking dow...


In [10]:
doc_pd.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(doc_pd))
doc_pd.head(50)

,input,output,instruction
0,"Zest Airways, Inc. operated as AirAsia Zest (f...",(Asian Spirit | country | Republic of the Phil...,Assume you're highly competent in scrutinizing...
1,The short-beaked common dolphin (Delphinus del...,,"Suppose you're an expert in text dissection, a..."
2,Niklas Bergqvist (born 6 October 1962 in Stock...,(Niklas Bergqvist | date of birth | 6 October ...,Picture yourself as adept at understanding and...
3,Pacific Fair is a major shopping centre in Bro...,(Coles | country | Australia)\n(Toys ' R ' Us ...,Pretend that you are great at parsing a text a...
4,The Ministry of Home Security was a British go...,(Home Office | country | British)\n(Herbert Mo...,Convert the following text into semantic tripl...
5,The Lakes is a locality in Western Australia w...,(Shire of Northam | located in the administrat...,Assume you're highly competent in scrutinizing...
6,Alton is a city on the Mississippi River in Ma...,(Illinois | located in the administrative terr...,Pretend you are highly skilled in breaking dow...
7,Us and Them is the fifth studio album by Engli...,(Love and Hate in Dub | performer | Godflesh)\...,Convert the following text into semantic tripl...
8,"Nicola Lake is a glacially formed narrow, deep...",(British Columbia | located in the administrat...,Convert the following text into semantic tripl...
9,"Jonas Geirnaert (born July 28, 1982) studied a...",(BASTA | cast member | Geirnaert)\n(KASK | loc...,Assume you're highly competent in scrutinizing...


In [11]:
ttt_pd.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(ttt_pd))
ttt_pd.head(50)

,instruction,input,output,original_output
0,"Suppose you're an expert in text dissection, a...","The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Pretend you are highly skilled in breaking dow...,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,"Imagine you're an expert at deciphering text, ...",Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Pretend you are highly skilled in breaking dow...,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Think of yourself as efficient in deconstructi...,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
5,Picture yourself as adept at understanding and...,Aktieselskab is the operating organisation for...,(Aarhus_Airport | operatingOrganisation | Akti...,['Aarhus_Airport | operatingOrganisation | Akt...
6,Convert the following text into semantic tripl...,Aarhus Airport runway length is 2776.0.,(Aarhus_Airport | runwayLength | 2776.0),['Aarhus_Airport | runwayLength | 2776.0']
7,Convert the following text into semantic tripl...,The runway length at Aarhus Airport is 2777.0 ...,(Aarhus_Airport | runwayLength | 2777.0),['Aarhus_Airport | runwayLength | 2777.0']
8,"Suppose you're a maestro in textual analysis, ...",Aarhus Airport runway name is 10L/28R.,"(Aarhus_Airport | runwayName | ""10L/28R"")","['Aarhus_Airport | runwayName | ""10L/28R""']"
9,"Suppose you're a maestro in textual analysis, ...","The runway name at Aarhus Airport is ""10R/28L"".","(Aarhus_Airport | runwayName | ""10R/28L"")","['Aarhus_Airport | runwayName | ""10R/28L""']"


In [14]:
ske_val.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(ske_val))
ske_val.head(50)

,instruction,input,output,original_output
0,"Suppose you're an expert in text dissection, a...",查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥...,(查尔斯·阿兰基斯 | 出生地 | 圣地亚哥)\n(查尔斯·阿兰基斯 | 出生日期 | 19...,"['查尔斯·阿兰基斯 | 出生地 | 圣地亚哥', '查尔斯·阿兰基斯 | 出生日期 | 1..."
1,Pretend you are highly skilled in breaking dow...,《离开》是由张宇谱曲，演唱,(离开 | 歌手 | 张宇)\n(离开 | 作曲 | 张宇),"['离开 | 歌手 | 张宇', '离开 | 作曲 | 张宇']"
2,"Imagine you're an expert at deciphering text, ...",《愤怒的唐僧》由北京吴意波影视文化工作室与优酷电视剧频道联合制作，故事以喜剧元素为主，讲述唐...,(愤怒的唐僧 | 出品公司 | 北京吴意波影视文化工作室)\n(愤怒的唐僧 | 导演 | 吴意波),"['愤怒的唐僧 | 出品公司 | 北京吴意波影视文化工作室', '愤怒的唐僧 | 导演 | ..."
3,"Suppose you're an expert in text dissection, a...",李治即位后，萧淑妃受宠，王皇后为了排挤萧淑妃，答应李治让身在感业寺的武则天续起头发，重新纳入后宫,(李治 | 妻子 | 萧淑妃)\n(萧淑妃 | 丈夫 | 李治),"['李治 | 妻子 | 萧淑妃', '萧淑妃 | 丈夫 | 李治']"
4,Pretend you are highly skilled in breaking dow...,《工业4 . 0》是2015年机械工业出版社出版的图书，作者是（德）阿尔冯斯·波特霍夫，恩斯...,(工业4.0 | 作者 | 恩斯特·安德雷亚斯·哈特曼)\n(工业4.0 | 作者 | 阿尔...,"['工业4.0 | 作者 | 恩斯特·安德雷亚斯·哈特曼', '工业4.0 | 作者 | 阿..."
5,Picture yourself as adept at understanding and...,周佛海被捕入狱之后，其妻杨淑慧散尽家产请蒋介石枪下留人，于是周佛海从死刑变为无期，不过此人或...,(周佛海 | 妻子 | 杨淑慧)\n(杨淑慧 | 丈夫 | 周佛海),"['周佛海 | 妻子 | 杨淑慧', '杨淑慧 | 丈夫 | 周佛海']"
6,Assume you're highly competent in scrutinizing...,《李烈钧自述》是2011年11月1日人民日报出版社出版的图书，作者是李烈钧,(李烈钧自述 | 作者 | 李烈钧)\n(李烈钧自述 | 出版社 | 人民日报出版社),"['李烈钧自述 | 作者 | 李烈钧', '李烈钧自述 | 出版社 | 人民日报出版社']"
7,"As someone proficient in text parsing, you sho...",除演艺事业外，李冰冰热心公益，发起并亲自参与多项环保慈善活动，积极投身其中，身体力行担起了回...,(少年包青天 | 主演 | 李冰冰),['少年包青天 | 主演 | 李冰冰']
8,"Suppose you're an expert in text dissection, a...",马志舟，1907年出生，陕西三原人，汉族，中国共产党，任红四团第一连连长，1907年逝世,(马志舟 | 国籍 | 中国)\n(马志舟 | 出生日期 | 1907)\n(马志舟 | 民...,"['马志舟 | 国籍 | 中国', '马志舟 | 出生日期 | 1907', '马志舟 | ..."
9,Convert the following text into semantic tripl...,斑刺莺是雀形目、剌嘴莺科的一种动物，分布于澳大利亚和新西兰，包括澳大利亚、新西兰、塔斯马尼亚...,(斑刺莺 | 目 | 雀形目),['斑刺莺 | 目 | 雀形目']


In [12]:
# instructions_base_long = [
#     "Derive semantic triplets from the subsequent text and provide a step by step explanation.",
#     "Retrieve triplets from the following text and describe each step.",
#     "Pull out triplets from the text underneath and provide a thorough breakdown of the process.",
#     "Draw out semantic triplets from the text provided below and outline the process step by step.",
#     "Elicit triplets from the subsequent textual content and describe the process.",
#     "Generate semantic triplets from the text beneath and elaborate on each step of the process.",
#     "Obtain triplets from the text that follows and describe every step in detail.",
#     "Bring out triplets from the text that's below and detail the entire process systematically.",
#     "Convert the following text into semantic triplets give a stepwise explanation.",
#     "Turn the text below into triplets and walk me through the procedure.",
#     "Transform the subsequent text into semantic triplets and explain the process step by step.",
#     "Transpose the text underneath into triplets and elaborate on the procedure.",
#     "Reformat the following text into semantic triplets and guide me through each stage.",
#     "Process the text below into triplets and present a stepwise analysis.",
#     "Interpret the subsequent text as semantic triplets and explain step by step.",
#     "Decipher the text underneath into triplets and outline each step.",
#     "Transcribe the following text into semantic triplets and lay out the procedure in order.",
#     "Create triplets from the text below and offer a comprehensive guide through the steps.",
#     "Alter the text that follows into semantic triplets and illustrate every step involved.",
#     "Construct semantic triplets from the text provided below and explain the process.",
# ]
instructions_base_long = [
    "Convert the following text into semantic triplets and provide a step by step explanation.",
    "Imagine you're an expert at deciphering text, capable of pulling out all entities and their relationships. When provided a text, your task is to identify all possible triplets, following the format: (subject | relationship | object). Additionally, provide a step by step explanation.",
    "As someone proficient in text parsing, you should be able to identify all entities and their associated relationships from a given text. I will hand you a text, and your job is to derive all potential triples, using the structure: (subject | relationship | object). Futhermore, provide a thorough breakdown of the process.",  
    "Envision yourself as skilled in analyzing text to the point of being able to draw out all entities and their correlations. Once I give you a text, your assignment is to generate all the plausible triples, formatted as: (subject | relationship | object). Offer a comprehensive guide through the steps.",
    "Assume you're highly competent in scrutinizing a piece of text and successfully distilling all its entities along with their connections. I'll provide a text, and you are to extract every possible triplet, following the convention: (subject | relationship | object). Detail the entire process systematically.",
    "Picture yourself as adept at understanding and dissecting text, capable of efficiently identifying all entities and their interrelationships. Given a text, it's your task to isolate all potential triples, in the format of: (subject | relationship | object). Ensure that you also lay out the procedure used.",
    "Suppose you're an expert in text dissection, able to accurately pull out all entities and their ties from any text. I will provide you with a text, and your task is to excavate all conceivable triples, following the template: (subject | relationship | object). Describe every step in the triple generation process in detail.",
    "Pretend you are highly skilled in breaking down a text, with the capability to effectively discern all the entities and their interactions. Once I supply you with a text, your mission is to extract all potential triple structures, in the form of: (subject | relationship | object). Please also elaborate on each step of the process.",
    "Think of yourself as efficient in deconstructing a text and precisely identifying all the entities and their interrelations. I'll furnish you with a text and your job is to gather all potential triples, adhering to the pattern: (subject | relationship | object). In addition to generating the triples, outline each step of the process.",
    "Suppose you're a maestro in textual analysis, capable of efficiently extracting all entities and their associations. Upon receiving a text from me, it's your duty to produce all possible triples, consistent with the arrangement: (subject | relationship | object). Guide me through each stage of the generation process.",
    "Pretend that you are great at parsing a text and can effectively extract all entities and their relationships from a text. I will give you a text and you extract all the possible triples.  The output format is: (subject | relationship | object). Ensure you elaborate on each step of the process.",
]

In [23]:
mod = [0, 2, 4, 5, 8, 9, 11, 12, 13, 14, 15, 16, 18, 23, 24, 25, 31, 35, 39, 40, 41, 45, 46, 53, 54, 61, 62, 64, 69, 70, 71, 73, 74, 78, 79, 80, 84, 85, 89, 93, 94, 96, 102, 104, 105, 106, 107, 108, 110, 117, 118, 119, 120, 121, 123, 124, 125, 131, 132, 133, 134, 135, 139, 140, 141, 145, 149, 150, 151, 152, 154, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 176, 180, 181, 184, 188, 192, 193, 195, 198, 199, 207, 216, 217, 224, 225, 226, 227, 229, 243, 244, 246, 247, 251, 258, 263, 264, 265, 267, 268, 270, 271, 279, 280, 281, 282, 288, 293, 294, 295, 298, 300, 301, 302, 306, 307, 308, 318, 320, 321, 322, 323, 324, 325, 326, 327, 335, 337, 339, 343, 352, 357, 358, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 381, 393, 400, 401, 402, 410, 411, 412, 422, 425, 426, 427, 428, 436, 437, 438, 439, 440, 441, 442, 457, 459, 460, 473, 474, 475, 476, 477, 478, 479, 486, 487, 488, 489, 490, 491, 492, 493, 494, 
495, 496, 497, 498, 499, 500, 501, 502, 503, 507, 508, 509, 510, 511, 512, 532, 577, 578, 581, 584, 585, 586, 587, 588, 590, 591, 594, 596, 598, 600, 602, 606, 608, 610, 613, 620, 627, 628, 630, 634, 636, 637, 639, 642, 644, 646, 658, 668, 669, 677, 678, 706, 720, 722, 723, 729, 735, 747, 748, 777, 778, 784, 806, 818, 844, 882, 888, 896, 916, 917, 918, 919, 937, 941, 947, 952, 955, 964, 965, 971, 980, 992, 1003, 1009, 1010, 1015, 1018, 1022, 1025, 1035, 1036, 1050, 1056, 1057, 1058, 1059, 1064, 1065, 1067, 1080, 1082, 1083, 1088, 1089, 1099, 1139, 1140, 1141, 1166, 1169, 1177, 1184, 1195, 1196, 1197, 1203, 1204, 1210, 1226, 
1237, 1241, 1253, 1268, 1271, 1278, 1285, 1301, 1328, 1338, 1341, 1342, 1343, 1346, 1347, 1350, 1351, 1352, 1358, 1359, 1360, 1361, 1363, 1364, 1365, 1366, 1367, 1368, 1370, 1371, 1374, 1375, 1383, 1384, 1385, 1386, 1388, 1389, 1392, 1393, 1394, 1397, 1398, 1402, 1403, 1406, 1407, 1408, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1429, 1430, 1431, 1435, 1436, 1442, 1446, 1447, 1448, 1450, 1451, 1453, 1454, 1457, 1458, 1459, 1460, 1465, 1472, 1473, 1474, 1477, 1480, 
1483, 1484, 1485, 1486, 1490, 1491, 1494, 1495, 1496, 1508, 1511, 1512, 1529, 1536, 1537, 1543, 1581, 1582, 1583, 1584, 1585, 1587, 1588, 1589, 1590, 1591, 1592, 1595, 1596, 1597, 1601, 1602, 1604, 1605, 1606, 1607, 1608, 1632, 1633, 1638, 1645, 1672, 1673, 1682, 1697, 1719, 1723, 1738, 1740, 1742, 1743, 1762, 1773, 1775, 1776, 1777, 1781, 1785, 1786, 1787, 1789, 1792, 1793, 1798, 1801, 1803, 1804, 1805, 1806, 1807, 1808, 1813, 1814, 1816, 1817, 1819, 1821, 1822, 1825, 
1826, 1827, 1830, 1831, 1837, 1838, 1839, 1840, 1841, 1846, 1847, 1849, 1850, 1851, 1852, 1854, 1858, 1859, 1864, 1865, 1878, 1881, 1882, 1883, 1884, 1888, 1889, 1891, 1892, 1896, 1898, 1899, 1900, 1904, 1905, 1907, 1910, 1913, 1914, 1920, 1921, 1922, 1923, 1924, 1926, 1928, 1933, 1939, 1942, 1943, 1944, 1951, 1987, 1988, 1997, 2004, 2046, 2068, 2094, 2095, 2096, 2097, 2137, 2154, 2173, 2179, 2184, 2220, 2226, 2285, 2332, 2336, 2338, 2340, 2344, 2345, 2346, 2347, 2348, 
2395, 2409, 2418, 2436, 2440, 2456, 2471, 2525, 2532, 2544, 2556, 2573, 2587, 2588, 2619, 2620, 2630, 2631, 2681, 2706, 2730, 2739, 2783, 2787, 2788, 2791, 2793, 2800, 2831, 2874, 2918, 2923, 2924, 2929, 2936, 2939, 2946, 2947, 2948, 2950, 2956, 2959, 2960, 2966, 2967, 2968, 2969, 2974, 2977, 2978, 2982, 2984, 2992, 2999, 3006, 3007, 3008, 3009, 3013, 3014, 3015, 3020, 3021, 3024, 3026, 3028, 3029, 3033, 3034, 3036, 3037, 3040, 3041, 3045, 3048, 3050, 3053, 3057, 3059, 
3061, 3062, 3063, 3066, 3068, 3070, 3071, 3073, 3075, 3078, 3079, 3083, 3084, 3085, 3088, 3090, 3091, 3092, 3093, 3095, 3096, 3102, 3103, 3104, 3107, 3108, 3109, 3115, 3117, 3119, 3121, 3123, 3127, 3129, 3131, 3132, 3133, 3134, 3140, 3148, 3153, 3168, 3197, 3198, 3199, 3200, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3210, 3211, 3212, 3213, 3215, 3216, 3219, 3224, 3227, 3229, 3230, 3231, 3233, 3234, 3243, 3245, 3246, 3247, 3251, 3252, 3257, 3258, 3259, 3260, 3261, 
3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3271, 3275, 3276, 3285, 3286, 3288, 3289, 3290, 3292, 3294, 3295, 3296, 3297, 3298, 3304, 3307, 3308, 3309, 3315, 3316, 3317, 3318, 3319, 3320, 3322, 3325, 3326, 3327, 3328, 3330, 3331, 3332, 3333, 3334, 3335, 3336, 3337, 3339, 3340, 3341, 3342, 3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351, 3353, 3354, 3355, 3356, 3357, 3362, 3364, 3365, 3366, 3367, 3368, 3371, 3372, 3373, 3374, 3375, 3377, 3378, 3379, 3380, 
3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3392, 3393, 3406, 3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3439, 3440, 3445, 3446, 3447, 3448, 3451, 3456, 3457, 3461, 3467, 3468, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3479, 3480, 3481, 3482, 3483, 3487, 3489, 3491, 3494, 3497, 3500, 3501, 3502, 3508, 3515, 3518, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3548, 3553, 3557, 3558, 
3559, 3560, 3561, 3562, 3563, 3564, 3576, 3577, 3578, 3579, 3580, 3581, 3582, 3583, 3584, 3585, 3586, 3587, 3588, 3589, 3590, 3591, 3592, 3593, 3594, 3606, 3607, 3614, 3615, 3618, 3619, 3620, 3621, 3625, 3627, 3628, 3629, 3633, 3634, 3635, 3638, 3639, 3641, 3643, 3645, 3646, 3647, 3652, 3653, 3656, 3657, 3659, 3662, 3718, 3719, 3723, 3724, 3725, 3726, 3727, 3733, 3766, 3798, 3832, 3833, 3848, 3854, 3857, 3872, 3873, 3876, 3877, 3885, 3887, 3889, 3890, 3897, 3898, 3899, 
3914, 3924, 3925, 3933, 3934, 3935, 3938, 3947, 3952, 3954, 3959, 3975, 3977, 3980, 3981, 3988, 3991, 3995, 3996, 3999, 4003, 4004, 4005, 4069, 4150, 4154, 4155, 4156, 4157, 4158, 4160, 4161, 4163, 4166, 4171, 4172, 4173, 4174, 4182, 4183, 4185, 4186, 4195, 4196, 4197, 4198, 4202, 4204, 4205, 4206, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 4216, 4217, 4218, 4219, 4222, 4223, 4224, 4225, 4226, 4227, 4228, 4229, 4230, 4231, 4232, 4233, 4234, 4235, 4237, 4238, 4239, 4244, 
4245, 4246, 4247, 4248, 4250, 4256, 4257, 4260, 4262, 4263, 4266, 4267, 4269, 4270, 4271, 4272, 4274, 4275, 4279, 4280, 4282, 4283, 4284, 4285, 4286, 4287, 4288, 4289, 4290, 4291, 4292, 4293, 4294, 4295, 4296, 4297, 4298, 4299, 4300, 4303, 4304, 4305, 4306, 4307, 4309, 4311, 4314, 4333, 4334, 4360, 4361, 4362, 4363, 4364, 4366, 4368, 4369, 4370, 4375, 4488, 4489, 4490, 4491, 4492, 4493, 4503, 4504, 4505, 4506, 4507, 4508, 4509, 4510, 4511, 4516, 4517, 4518, 4519, 4520, 
4521, 4522, 4523, 4526, 4527, 4529, 4535, 4536, 4537, 4538, 4539, 4540, 4541, 4542, 4547, 4548, 4549, 4550, 4551, 4552, 4553, 4554, 4555, 4556, 4557, 4558, 4559, 4560, 4561, 4562, 4563, 4564, 4565, 4567, 4568, 4569, 4570, 4571, 4572, 4573, 4574, 4580, 4581, 4582, 4583, 4584, 4585, 4586, 4587, 4588, 4589, 4592, 4594, 4595, 4596, 4597, 4598, 4599, 4600, 4601, 4602, 4603, 4604, 4605, 4606, 4607, 4608, 4609, 4610, 4611, 4612, 4613, 4614, 4615, 4616, 4617, 4618, 4619, 4620, 
4621, 4622, 4623, 4624, 4626, 4627, 4628, 4629, 4630, 4631, 4632, 4633, 4634, 4635, 4636, 4637, 4638, 4639, 4640, 4641, 4644, 4645, 4646, 4647, 4648, 4649, 4650, 4653, 4654, 4655, 4656, 4658, 4661, 4662, 4663, 4664, 4665, 4666, 4667, 4668, 4669, 4670, 4671, 4672, 4673, 4674, 4675, 4676, 4678, 4679, 4680, 4681, 4682, 4683, 4684, 4685, 4686, 4687, 4690, 4691, 4692, 4693, 4694, 4695, 4699, 4700, 4701, 4702, 4703, 4704, 4705, 4706, 4707, 4708, 4709, 4711, 4712, 4713, 4714, 
4716, 4717, 4718, 4719, 4720, 4721, 4722, 4723, 4724, 4728, 4729, 4730, 4735, 4737, 4738, 4739, 4740, 4741, 4742, 4743, 4745, 4746, 4747, 4748, 4749, 4750, 4751, 4752, 4754, 4756, 4757, 4758, 4759, 4760, 4763, 4764, 4794, 4800, 4802, 4812, 4823, 4824, 4840, 4841, 4842, 4869, 4877, 4878, 4889, 4890, 4891, 4913, 4920, 4977, 4982, 4983, 4984, 4986, 4987, 4988, 4989, 4990, 4991, 4992, 4993, 4994, 4997, 4999, 5000, 5026, 5042, 5043, 5044, 5045, 5046, 5049, 5056, 5057, 5058, 
5059, 5060, 5061, 5062, 5063, 5068, 5069, 5070, 5071, 5072, 5076, 5082, 5091, 5094, 5095, 5110, 5117, 5118, 5119, 5120, 5134, 5138, 5169, 5170, 5171, 5179, 5183, 5188, 5189, 5190, 5191, 5201, 5202, 5212, 5213, 5220, 5221, 5222, 5224, 5232, 5242, 5277, 5332, 5339, 5340, 5370, 5371, 5348, 5430, 5433, 5434, 5436, 5439, 5442, 5443, 5444, 5445, 5448, 5454, 5455, 5456, 5457, 5458, 5459, 5460, 5461, 5462, 5463, 5464, 5465, 5466, 5467, 5468, 5469, 5475, 5476, 5479, 5480, 5481, 
5482, 5483, 5484, 5485, 5486, 5490, 5495, 5496, 5497, 5498, 5500, 5502, 5503, 5513, 5520, 5521, 5524, 5525, 5528, 5529, 5532, 5533, 5534, 5539, 5540, 5544, 5545, 5546, 5547, 5550, 5551, 5552, 5561, 5562, 5564, 5565, 5570, 5571, 5572, 5574, 5575, 5578, 5581, 5582, 5583, 5585, 5586, 5587, 5590, 5591, 5592, 5593, 5595, 5596, 5597, 5600, 5602, 5603, 5606, 5607, 5614, 5615, 5617, 5619, 5620, 5621, 5627, 5628, 5629, 5630, 5631, 5632, 5633, 5634, 5636, 5673, 5674, 5675, 5676, 
5677, 5678, 5679, 5680, 5681, 5682, 5683, 5684, 5685, 5686, 5687, 5688, 5689, 5690, 5691, 5692, 5693, 5694, 5695, 5696, 5697, 5698, 5699, 5700, 5701, 5702, 5703, 5704, 5705, 5706, 5707, 5708, 5709, 5710, 5711, 5712, 5714, 5715, 5716, 5717, 5718, 5720, 5721, 5722, 5723, 5724, 5725, 5726, 5727, 5728, 5729, 5730, 5731, 5732, 5733, 5734, 5735, 5736, 5737, 5738, 5739, 5740, 5741, 5742, 5743, 5744, 5745, 5746, 5747, 5748, 5749, 5750, 5751, 5752, 5753, 5754, 5755, 5758, 5759, 
5762, 5763, 5764, 5765, 5766, 5767, 5768, 5777, 5778, 5779, 5781, 5782, 5783, 5789, 5790, 5791, 5792, 5793, 5794, 5795, 5796, 5797, 5798, 5799, 5800, 5801, 5802, 5803, 5804, 5805, 5806, 5807, 5808, 5809, 5810, 5811, 5812, 5813, 5814, 5815, 5816, 5819, 5822, 5823, 5824, 5826, 5827, 5828, 5830, 5831, 5832, 5833, 5838, 5841, 5842, 5843, 5844, 5845, 5846, 5847, 5848, 5489, 5850, 5851, 5852, 5853, 5854, 5855, 5856, 5857, 5858, 5873, 5874, 5875, 5876, 5877, 5878, 5879, 5880, 
5881, 5882, 5883, 5884, 5885, 5886, 5887, 5888, 5889, 5890, 5891, 5892, 5893, 5894, 5902, 5903, 5904, 5905, 5906, 5912, 5915, 5921, 5923, 5927, 5928, 5932, 5933, 5934, 5935, 5936, 5937, 5938, 5939, 5940, 5941, 5946, 5947, 5950, 5951, 5952, 5953, 5954, 5955, 5956, 5960, 5961, 5962, 5963, 5964, 5965, 5966, 5967, 5968, 5969, 5970, 5971, 5972, 5973, 5990, 5991, 5992, 5993, 5994, 5995, 5996, 5997, 5998, 6000, 6001, 6018, 6019, 6020, 6022, 6023, 6024, 6028, 6029, 6030, 6031, 
6032, 6033, 6034, 6035, 6046, 6047, 6048, 6049, 6050, 6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6061, 6062, 6063, 6064, 6065, 6066, 6067, 6068, 6069, 6070, 6071, 6072, 6073, 6076, 6080, 6081, 6082, 6083, 6084, 6085, 6086, 6087, 6088, 6089, 6090, 6100, 6101, 6102, 6103, 6104, 6106, 6107, 6110, 6111, 6112, 6113, 6114, 6115, 6116, 6117, 6119, 6120, 6121, 6122, 6123, 6124, 6126, 6127, 6129, 6130, 6131, 6132, 6133, 6134, 6135, 6136, 6137, 6139, 6141, 6144, 
6147, 6148, 6150, 6151, 6152, 6153, 6154, 6155, 6156, 6157, 6158, 6159, 6160, 6161, 6162, 6163, 6169, 6170, 6171, 6172, 6175, 6176, 6177, 6178, 6179, 6184, 6185, 6186, 6213, 6214, 6215, 6216, 6220, 6222, 6226, 6230, 6233, 6234, 6238, 6239, 6240, 6241, 6242, 6243, 6244, 6249, 6250, 6293, 6333, 6334, 6335, 6351, 6352, 6353, 6354, 6355, 6356, 6357, 6380, 6388, 6390, 6404, 6408, 6409, 6410, 6411, 6412, 6423, 6426, 6427, 6430, 6432, 6433, 6434, 6435, 6436, 6437, 6438, 6448, 
6450, 6463, 6464, 6465, 6466, 6467, 6468, 6469, 6470, 6473, 6475, 6476, 6478, 6480, 6482, 6483, 6489, 6490, 6494, 6496, 6501, 6527, 6528, 6529, 6530, 6531, 6540, 6541, 6542, 6543, 6544, 6545, 6546, 6547, 6548, 6549, 6550, 6551, 6552, 6553, 6560, 6563, 6623, 6628, 6643, 6647, 6648, 6712, 6713, 6714, 6715, 6716, 6717, 6718, 6719, 6720, 6721, 6722, 6723, 6724, 6725, 6726, 6727, 6728, 6729, 6730, 6731, 6732, 6733, 6734, 6735, 6736, 6737, 6742, 6743, 6744, 6745, 6746, 6747, 
6748, 6750, 6753, 6754, 6755, 6756, 6757, 6758, 6759, 6760, 6761, 6762, 6763, 6764, 6765, 6766, 6767, 6768, 6769, 6770, 6771, 6772, 6773, 6774, 6775, 6776, 6777, 6778, 6779, 6780, 6781, 6784, 6785, 6786, 6787, 6788, 6789, 6790, 6791, 6792, 6793, 6794, 6795, 6796, 6797, 6798, 6799, 6800, 6801, 6802, 6803, 6804, 6805, 6806, 6807, 6808, 6812, 6813, 6814, 6815, 6816, 6817, 6818, 6819, 6820, 6821, 6822, 6823, 6826, 6827, 6829, 6830, 6831, 6832, 6833, 6834, 6835, 6836, 6838, 
6839, 6840, 6841, 6843, 6844, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855, 6856, 6857, 6858, 6864, 6865, 6866, 6867, 6868, 6869, 6870, 6871, 6872, 6873, 6874, 6875, 6876, 6877, 6878, 6879, 6880, 6881, 6882, 6883, 6884, 6885, 6886, 6887, 6888, 6889, 6890, 6891, 6892, 6893, 6894, 6895, 6897, 6900, 6904, 6905, 6906, 6907, 6911, 6912, 6917, 6918, 6919, 6928, 6929, 6931, 6945, 6946, 6959, 6960, 6961, 6966, 6967, 6968, 6969, 6970, 6973, 6974, 6975, 6976, 6977, 6978, 
6979, 7028, 7057, 7067, 7087, 7088, 7089, 7090, 7091, 7092, 7093, 7094, 7099, 7100, 7101, 7102, 7103, 7104, 7105, 7106, 7107, 7111, 7112, 7113, 7114, 7115, 7116, 7117, 7118, 7119, 7120, 7121, 7122, 7123, 7124, 7125, 7126, 7127, 7128, 7129, 7130, 7131, 7132, 7133, 7134, 7135, 7136, 7137, 7138, 7139, 7140, 7141, 7142, 7143, 7144, 7145, 7146, 7147, 7148, 7149, 7150, 7151, 7152, 7153, 7154, 7155, 7156, 7157, 7158, 7159, 7160, 7162, 7163, 7164, 7165, 7166, 7167, 7168, 7169, 
7170, 7171, 7172, 7173, 7174, 7175, 7176, 7177, 7178, 7179, 7180, 7181, 7182, 7184, 7185, 7186, 7187, 7188, 7189, 7190, 7191, 7192, 7193, 7194, 7195, 7196, 7197, 7198, 7199, 7200, 7201, 7202, 7203, 7204, 7205, 7206, 7207, 7208, 7209, 7210, 7211, 7212, 7213, 7214, 7215, 7216, 7217, 7218, 7219, 7220, 7221, 7222, 7223, 7224, 7225, 7226, 7227, 7228, 7229, 7230, 7231, 7232, 7233, 7234, 7235, 7236, 7237, 7238, 7239, 7240, 7241, 7242, 7243, 7244, 7245, 7246, 7247, 7248, 7249, 
7250, 7251, 7252, 7253, 7255, 7256, 7257, 7258, 7259, 7260, 7261, 7262, 7263, 7264, 7265, 7266, 7267, 7268, 7269, 7270, 7272, 7273, 7274, 7275, 7276, 7277, 7278, 7279, 7280, 7281, 7282, 7283, 7284, 7285, 7286, 7287, 7288, 7289, 7290, 7291, 7292, 7293, 7294, 7295, 7296, 7297, 7298, 7299, 7300, 7301, 7302, 7303, 7304, 7305, 7306, 7307, 7308, 7309, 7310, 7311, 7312, 7313, 7314, 7315, 7316, 7317, 7319, 7320, 7321, 7322, 7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 
7332, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7398, 7399, 7440, 7442, 7447, 7449, 7464, 7467, 7468, 7510, 7517, 7519, 7536, 7537, 7538, 7539, 
7552, 7557, 7558, 7582, 7583, 7628, 7629, 7633, 7634, 7635, 7637, 7638, 7639, 7640, 7641, 7642, 7643, 7644, 7646, 7647, 7650, 7651, 7652, 7653, 7662, 7665, 7667, 7687, 7688, 7689, 7690, 7701, 7702, 7703, 7705, 7706, 7707, 7710, 7712, 7713, 7714, 7715, 7716, 7717, 7718, 7719, 7720, 7721, 7722, 7723, 7780, 7781, 7782, 7783, 7784, 7785, 7786, 7842, 7845, 7906, 7907, 7908, 7909, 7910, 7911, 7912, 7913, 7914, 7950, 7970, 8007, 8063, 8065, 8090, 8093, 8100, 8101, 8102, 8103, 
8104, 8105, 8106, 8108, 8114, 8116, 8118, 8119, 8120, 8122, 8123, 8124, 8126, 8127, 8128, 8129, 8130, 8132, 8135, 8136, 8137, 8143, 8144, 8145, 8146, 8147, 8148, 8149, 8150, 8151, 8154, 8155, 8156, 8158, 8159, 8171, 8172, 8173, 8174, 8175, 8176, 8177, 8178, 8179, 8180, 8181, 8182, 8183, 8186, 8187, 8188, 8189, 8190, 8191, 8192, 8193, 8195, 8198, 8200, 8201, 8213, 8214, 8216, 8217, 8218, 8219, 8220, 8221, 8222, 8223, 8224, 8225, 8226, 8227, 8228, 8233, 8234, 8235, 8236, 
8238, 8239, 8240, 8241, 8242, 8243, 8245, 8254, 8255, 8259, 8260, 8261, 8262, 8263, 8265, 8267, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277, 8279, 8280, 8281, 8282, 8283, 8284, 8285, 8286, 8287, 8288, 8289, 8290, 8291, 8292, 8293, 8294, 8295, 8296, 8297, 8298, 8299, 8302, 8303, 8304, 8306, 8314, 8315, 8316, 8317, 8318, 8319, 8320, 8326, 8327, 8328, 8329, 8330, 8331, 8332, 8334, 8336, 8337, 8338, 8343, 8344, 8345, 8351, 8380, 8402, 8403, 8404, 8405, 8406, 
8407, 8408, 8409, 8410, 8411, 8412, 8413, 8414, 8415, 8416, 8417, 8418, 8419, 8420, 8421, 8422, 8423, 8424, 8425, 8426, 8427, 8428, 8429, 8430, 8431, 8432, 8433, 8434, 8435, 8436, 8437, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8448, 8449, 8450, 8451, 8452, 8453, 8454, 8455, 8456, 8457, 8458, 8459, 8460, 8461, 8462, 8463, 8464, 8465, 8466, 8467, 8468, 8469, 8470, 8471, 8472, 8473, 8474, 8475, 8476, 8477, 8478, 8479, 8480, 8481, 8482, 8483, 8484, 8485, 
8486, 8487, 8488, 8489, 8492, 8493, 8494, 8495, 8496, 8497, 8498, 8499, 8500, 8501, 8502, 8503, 8504, 8505, 8506, 8507, 8518, 8519, 8520, 8521, 8522, 8523, 8524, 8525, 8526, 8527, 8528, 8529, 8530, 8531, 8532, 8533, 8534, 8535, 8536, 8537, 8538, 8539, 8540, 8541, 8542, 8543, 8544, 8545, 8546, 8547, 8548, 8549, 8550, 8551, 8552, 8553, 8554, 8555, 8556, 8557, 8560, 8561, 8563, 8564, 8566, 8567, 8568, 8575, 8576, 8577, 8578, 8579, 8580, 8581, 8582, 8583, 8584, 8585, 8591, 
8592, 8596, 8597, 8598, 8601, 8602, 8603, 8604, 8605, 8606, 8607, 8608, 8609, 8610, 8611, 8612, 8613, 8614, 8615, 8622, 8623, 8624, 8625, 8626, 8627, 8628, 8629, 8630, 8631, 8632, 8633, 8634, 8635, 8636, 8637, 8638, 8639, 8646, 8647, 8648, 8656, 8659, 8660, 8664, 8668, 8672, 8673, 8674, 8675, 8676, 8677, 8678, 8679, 8680, 8681, 8682, 8683, 8684, 8685, 8688, 8691, 8692, 8693, 8695, 8696, 8697, 8698, 8699, 8701, 8704, 8705, 8706, 8707, 8708, 8709, 8710, 8712, 8713, 8714, 
8715, 8716, 8717, 8730, 8731, 8732, 8733, 8734, 8737, 8738, 8739, 8740, 8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758, 8759, 8765, 8766, 8767, 8768, 8769, 8770, 8771, 8772, 8773, 8778, 8779, 8780, 8781, 8782, 8783, 8784, 8786, 8787, 8788, 8789, 8790, 8791, 8792, 8793, 8794, 8795, 8796, 8797, 8798, 8799, 8800, 8801, 8802, 8803, 8804, 8805, 8806, 8807, 8808, 8809, 8810, 8811, 8812, 8814, 8820, 8821, 8822, 8823, 8824, 8825, 8826, 8827, 8828, 8829, 8830, 8831, 8832, 8833, 
8834, 8835, 8836, 8837, 8838, 8848, 8849, 8850, 8851, 8852, 8853, 8854, 8855, 8856, 8857, 8858, 8859, 8860, 8861, 8862, 8863, 8864, 8865, 8866, 8867, 8868, 8870, 8871, 8873, 8874, 8875, 8876, 8877, 8878, 8879, 8880, 8882, 8884, 8885, 8886, 8887, 8888, 8889, 8890, 8891, 8892, 8893, 8894, 8895, 8896, 8897, 8899, 8900, 8902, 8903, 8904, 8906, 8907, 8908, 8909, 8910, 8911, 8915, 8916, 8917, 8918, 8919, 8920, 8921, 8922, 8923, 8924, 8925, 8926, 8927, 8928, 8929, 8947, 8948, 
8949, 8950, 8951, 8952, 8978, 8985, 8986, 9005, 9006, 9007, 9008, 9009, 9040, 9041, 9062, 9064, 9065, 9066, 9072, 9077, 9079, 9081, 9083, 9085, 9087, 9089, 9090, 9104, 9106, 9107, 9108, 9109, 9110, 9111, 9125, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9141, 9143, 9156, 9157, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9176, 9178, 9179, 9180, 9181, 9183, 9184, 9185, 9186, 9187, 9220, 9221, 9230, 9231, 9232, 9233, 9234, 9235, 9236, 9237, 9238, 9239, 
9240, 9241, 9242, 9243, 9244, 9245, 9253, 9254, 9256, 9264, 9265, 9322, 9323, 9326, 9344, 9345, 9347, 9402, 9403, 9404, 9405, 9406, 9407, 9408, 9409, 9410, 9411, 9412, 9413, 9414, 9415, 9416, 9417, 9418, 9419, 9420, 9421, 9422, 9423, 9424, 9245, 9426, 9427, 9428, 9429, 9430, 9431, 9432, 9433, 9434, 9435, 9436, 9437, 9438, 9439, 9440, 9441, 9442, 9443, 9444, 9445, 9446, 9447, 9448, 9449, 9451, 9453, 9454, 9455, 9456, 9457, 9458, 9459, 9460, 9461, 9464, 9465, 9466, 9467, 
9468, 9469, 9470, 9471, 9472, 9473, 9474, 9477, 9478, 9479, 9481, 9482, 9483, 9484, 9485, 9486, 9487, 9488, 9489, 9490, 9491, 9492, 9493, 9494, 9495, 9497, 9498, 9499, 9500, 9501, 9502, 9503, 9504, 9505, 9506, 9507, 9508, 9509, 9510, 9511, 9512, 9513, 9514, 9515, 9516, 9517, 9518, 9519, 9520, 9521, 9522, 9523, 9524, 9525, 9526, 9527, 9528, 9529, 9530, 9531, 9532, 9533, 9534, 9535, 9536, 9537, 9538, 9539, 9540, 9541, 9542, 9543, 9544, 9545, 9546, 9547, 9548, 9549, 9550, 
9551, 9552, 9553, 9554, 9555, 9556, 9557, 9558, 9559, 9560, 9561, 9562, 9563, 9564, 9565, 9566, 9567, 9568, 9569, 9570, 9571, 9572, 9573, 9574, 9575, 9576, 9577, 9578, 9579, 9580, 9581, 9582, 9583, 9584, 9585, 9586, 9590, 9591, 9592, 9593, 9594, 9597, 9598, 9602, 9603, 9606, 9608, 9609, 9610, 9611, 9612, 9613, 9614, 9615, 9616, 9617, 9618, 9620, 9621, 9623, 9624, 9625, 9626, 9627, 9628, 9629, 9630, 9631, 9632, 9633, 9634, 9635, 9636, 9637, 9638, 9646, 9647, 9670, 9695, 
9701, 9711, 9712, 9713, 9714, 9715, 9716, 9717, 9718, 9719, 9723, 9724, 9275, 9726, 9727, 9728, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739, 9740, 9741, 9742, 9743, 9745, 9746, 9748, 9749, 9750, 9753, 9754, 9755, 9756, 9757, 9758, 9759, 9760, 9761, 9762, 9763, 9764, 9765, 9766, 9767, 9768, 9769, 9770, 9771, 9772, 9773, 9774, 9775, 9776, 9777, 9778, 9779, 9780, 9781, 9782, 9783, 9784, 9785, 9786, 9787, 9788, 9789, 9790, 9791, 9792, 9793, 9794, 9795, 9796, 
9797, 9798, 9799, 9800, 9801, 9802, 9803, 9804, 9805, 9806, 9807, 9808, 9809, 9810, 9811, 9812, 9813, 9814, 9815, 9816, 9817, 9818, 9819, 9820, 9821, 9822, 9823, 9824, 9825, 9826, 9827, 9828, 9829, 9830, 9831, 9832, 9833, 9834, 9835, 9836, 9837, 9838, 9839, 9840, 9841, 9842, 9843, 9844, 9845, 9846, 9847, 9848, 9849, 9850, 9851, 9852, 9853, 9854, 9855, 9856, 9857, 9858, 9859, 9860, 9861, 9862, 9863, 9864, 9865, 9866, 9867, 9868, 9869, 9872, 9873, 9874, 9875, 9876, 9877, 
9878, 9879, 9880, 9881, 9882, 9883, 9884, 9885, 9886, 9887, 9888, 9889, 9890, 9891, 9892, 9893, 9894, 9895, 9896, 9897, 9898, 9899, 9900, 9901, 9902, 9903, 9904, 9905, 9906, 9907, 9908, 9909, 9910, 9911, 9912, 9913, 9914, 9915, 9916, 9917, 9918, 9919, 9920, 9921, 9922, 9923, 9924, 9925, 9926, 9927, 9928, 9929, 9930, 9931, 9932, 9933, 9934, 9935, 9936, 9937, 9938, 9939, 9940, 9941, 9942, 9944, 9945, 9946, 9947, 9948, 9949, 9950, 9951, 9952, 9953, 9954, 9955, 9956, 9957, 
9958, 9959, 9960, 9961, 9962, 9963, 9964, 9965, 9966, 9967, 9968, 9969, 9970, 9971, 9972, 9975, 9976, 9977, 9978, 9979, 9980, 9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9989, 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 10023, 10024, 10055, 10056, 10058, 10064, 10065, 10066, 10071, 10072, 10109, 10110, 10112, 10116, 10118, 10120, 10121, 10133, 10134, 10135, 10136, 10137, 10154, 10155, 10173, 10176, 10203, 10204, 10205, 10241, 10242, 10243, 10245, 
10247, 10268, 10269, 10270, 10271, 10273, 10275, 10276, 10277, 10278, 10279, 10280, 10281, 10282, 10283, 10284, 10285, 10286, 10287, 10288, 10289, 10290, 10291, 10292, 10293, 10294, 10295, 10296, 10304, 10305, 10306, 10307, 10308, 10310, 10311, 10313, 10340, 10341, 10355, 10359, 10360, 10361, 10364, 10365, 10368, 10369, 10370, 10371, 10372, 10373, 10374, 10375, 10423, 10424, 10425, 10426, 10427, 10428, 10445, 10446, 10448, 10449, 10451, 10485, 10487, 10488, 10489, 10490, 10494, 10533, 10549, 10550, 10551, 10552, 10588, 10592, 10626, 10631, 10632, 10650, 10669, 10670, 10672, 10675, 10679, 10699, 10704, 10711, 10712, 10713, 10714, 10716, 10717, 10718, 10732, 10733, 10734, 10735, 10736, 10737, 10738, 10739, 10742, 10743, 10744, 10745, 10746, 10747, 10748, 10749, 10750, 10751, 10752, 10753, 10754, 10755, 10756, 10764, 10765, 10766, 10767, 10768, 10769, 10770, 10771, 10772, 10773, 10774, 10775, 10776, 10777, 10779, 10780, 10781, 10782, 10783, 10784, 10785, 10797, 10798, 10799, 10800, 10801, 10803, 10804, 10805, 10806, 10807, 10809, 10810, 10811, 10818, 10825, 10827, 10828, 10829, 10830, 10835, 
10836, 10838, 10841, 10842, 10843, 10844, 10848, 10849, 10850, 10851, 10852, 10853, 10854, 10855, 10856, 10857, 10858, 10859, 10860, 10861, 10862, 10863, 10864, 10865, 10866, 10867, 10868, 10869, 10870, 10871, 10872, 10873, 10874, 10875, 10876, 10877, 10878, 10884, 10885, 10886, 10887, 10888, 10889, 10890, 10892, 10893, 10894, 10895, 10896, 10897, 10898, 10901, 10902, 10903, 10904, 10905, 10906, 10907, 10908, 10909, 10910, 10911, 10912, 10915, 10916, 10923, 10924, 10925, 10928, 10929, 10930, 10931, 10932, 10934, 10937, 10948, 10949, 10950, 10951, 10952, 10954, 10955, 10956, 10957, 10958, 10959, 10960, 10961, 10962, 10963, 10964, 10965, 10966, 10967, 10968, 10969, 10970, 10971, 10972, 10973, 10974, 10975, 10976, 10977, 10978, 10979, 10980, 10981, 10982, 10983, 10984, 10985, 10986, 10987, 10988, 10989, 10990, 10991, 10992, 10993, 10994, 10995, 10996, 10997, 10998, 10999, 11000, 11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008, 11009, 11010, 11011, 11012, 11013, 11014, 11015, 11016, 11017, 11018, 11019, 11020, 11021, 11022, 11023, 11024, 11025, 11026, 11027, 11028, 11029, 11030, 11031, 
11032, 11033, 11034, 11035, 11036, 11037, 11038, 11039, 11040, 11041, 11042, 11043, 11044, 11045, 11046, 11047, 11048, 11049, 11050, 11051, 11052, 11053, 11054, 11055, 11056, 11057, 11058, 11059, 11060, 11071, 11072, 11073, 11074, 11075, 11076, 11077, 11078, 11079, 11080, 11081, 11082, 11083, 11084, 11085, 11086, 11087, 11088, 11089, 11090, 11091, 11092, 11093, 11094, 11095, 11096, 11097, 11098, 11099, 11100, 11101, 11102, 11103, 11104, 11109, 11110, 11111, 11112, 11113, 11114, 11115, 11121, 11122, 11123, 11124, 11125, 11126, 11127, 11128, 11129, 11130, 11131, 11132, 11137, 11138, 11139, 11140, 11141, 11142, 11143, 11144, 11145, 11146, 11147, 11148, 11149, 11150, 11151, 11152, 11153, 11154, 11155, 11158, 11163, 11164, 11165, 11166, 11167, 11168, 11169, 11170, 11171, 11172, 11173, 11174, 11175, 11176, 11177, 11178, 11179, 11182, 11183, 11184, 11186, 11187, 11188, 11189, 11191, 11192, 11193, 11194, 11195, 11198, 11199, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11211, 11212, 11216, 11217, 11218, 11221, 11222, 11223, 11225, 11227, 11228, 11229, 11230, 11231, 11232, 11233, 11234, 
11235, 11236, 11237, 11238, 11239, 11240, 11241, 11242, 11243, 11244, 11245, 11246, 11247, 11248, 11249, 11254, 11260, 11261, 11262, 11263, 11264, 11265, 11266, 11267, 11268, 11269, 11270, 11271, 11277, 11284, 11285, 11286, 11287, 11288, 11289, 11290, 11291, 11292, 11293, 11294, 11295, 11296, 11297, 11298, 11299, 11300, 11301, 11302, 11306, 11307, 11308, 11309, 11310, 11312, 11313, 11314, 11315, 11316, 11317, 11318, 11319, 11320, 11321, 11322, 11323, 11324, 11325, 11326, 11327, 11328, 11329, 11330, 11331, 11332, 11333, 11334, 11335, 11336, 11337, 11338, 11339, 11340, 11341, 11343, 11344, 11345, 11353, 11354, 11355, 11356, 11357, 11358, 11359, 11360, 11361, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11383, 11384, 11385, 11386, 11387, 11388, 11389, 11390, 11391, 11392, 11393, 11394, 11395, 11396, 11397, 11398, 11399, 11400, 11401, 11402, 11403, 11404, 11405, 11406, 11407, 11408, 11409, 11410, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11422, 11423, 11424, 11425, 11426, 11427, 11428, 11429, 11430, 11431, 11433, 11434, 11435, 11436, 
11437, 11439, 11441, 11442, 11443, 11444, 11445, 11447, 11448, 11449, 11450, 11451, 11452, 11453, 11455, 11456, 11457, 11458, 11459, 11460, 11461, 11462, 11463, 11464, 11465, 11466, 11467, 11468, 11472, 11473, 11494, 11495, 11496, 11508, 11511, 11512, 11513, 11514, 11515, 11516, 11517, 11528, 11529, 11530, 11531, 11536, 11538, 11545, 11546, 11547, 11548, 11549, 11550, 11551, 11552, 11558, 11559, 11560, 11561, 11562, 11563, 11564, 11565, 11566, 11567, 11568, 11569, 11570, 11571, 11572, 11579, 11580, 11596, 11597, 11601, 11602, 11603, 11604, 11605, 11606, 11607, 11608, 11609, 11610, 11611, 11612, 11613, 11614, 11615, 11616, 11617, 11618, 11619, 11620, 11645, 11646, 11647, 11648, 11649, 11650, 11651, 11652, 11653, 11654, 11655, 11656, 11657, 11666, 11667, 11669, 11670, 11671, 11673, 11737, 11779, 11780, 11781, 11782, 11783, 11784, 11785, 11786, 11787, 11788, 11789, 11790, 11791, 11792, 11793, 11794, 11795, 11796, 11797, 11798, 11799, 11800, 11801, 11802, 11804, 11805, 11806, 11807, 11808, 11809, 11810, 11811, 11812, 11813, 11814, 11815, 11816, 11817, 11818, 11819, 11820, 11821, 11822, 11823, 
11824, 11825, 11826, 11827, 11828, 11829, 11830, 11831, 11832, 11833, 11834, 11835, 11836, 11837, 11838, 11840, 11841, 11842, 11843, 11844, 11845, 11846, 11847, 11848, 11849, 11850, 11851, 11852, 11853, 11854, 11855, 11856, 11857, 11858, 11859, 11862, 11863, 11864, 11865, 11866, 11867, 11868, 11869, 11870, 11871, 11872, 11873, 11874, 11875, 11876, 11877, 11878, 11879, 11880, 11881, 11882, 11885, 11886, 11887, 11888, 11889, 11890, 11891, 11892, 11893, 11894, 11895, 11896, 11897, 11898, 11899, 11900, 11901, 11902, 11903, 11904, 11905, 11906, 11907, 11908, 11909, 11910, 11911, 11912, 11913, 11914, 11915, 11916, 11917, 11918, 11919, 11920, 11921, 11922, 11923, 11924, 11925, 11926, 11927, 11928, 11929, 11930, 11931, 11932, 11933, 11934, 11935, 11936, 11937, 11938, 11939, 11940, 11941, 11942, 11943, 11944, 11945, 11946, 11947, 11948, 11949, 11950, 11951, 11952, 11953, 11954, 11955, 11956, 11959, 11960, 11962, 11965, 11966, 11967, 11968, 11969, 11972, 11973, 11974, 11975, 11976, 11977, 11978, 11979, 11980, 11981, 11984, 11985, 11986, 11987, 11988, 11989, 11990, 11991, 11992, 11993, 11994, 11995, 
11996, 11997, 11998, 11999, 12000, 12001, 12002, 12003, 12004, 12005, 12006, 12007, 12008, 12009, 12010, 12011, 12012, 12013, 12042, 12053, 12054, 12055, 12056, 12058, 12059, 12060, 12061, 12064, 12065, 12066, 12067, 12068, 12069, 12070, 12071, 12072, 12073, 12074, 12075, 12076, 12077, 12078, 12079, 12080, 12081, 12082, 12083, 12084, 12085, 12086, 12087, 12088, 12089, 12090, 12091, 12092, 12093, 12094, 12095, 12096, 12097, 12098, 12099, 12100, 12101, 12102, 12103, 12104, 12105, 12106, 12107, 12108, 12109, 12110, 12111, 12112, 12113, 12114, 12115, 12116, 12117, 12118, 12119, 12120, 12121, 12122, 12123, 12124, 12125, 12126, 12127, 12128, 12129, 12130, 12131, 12132, 12133, 12134, 12135, 12136, 12137, 12138, 12139, 12140, 12141, 12142, 12143, 12144, 12145, 12146, 12147, 12148, 12149, 12150, 12151, 12152, 12153, 12154, 12157, 12158, 12159, 12160, 12161, 12162, 12163, 12164, 12165, 12166, 12167, 12168, 12169, 12170, 12171, 12172, 12174, 12175, 12176, 12177, 12178, 12179, 12180, 12181, 12182, 12183, 12184, 12185, 12186, 12187, 12188, 12189, 12190, 12191, 12192, 12193, 12194, 12195, 12196, 12197, 
12198, 12199, 12200, 12201, 12202, 12203, 12204, 12205, 12206, 12207, 12208, 12209, 12210, 12211, 12212, 12213, 12214, 12215, 12216, 12217, 12218, 12219, 12220, 12221, 12222, 12223, 12224, 12225, 12226, 12227, 12228, 12229, 12230, 12231, 12232, 12233, 12234, 12235, 12236, 12237, 12238, 12239, 12240, 12241, 12242, 12243, 12244, 12245, 12246, 12247, 12248, 12249, 12250, 12251, 12252, 12253, 12254, 12255, 12256, 12257, 12258, 12259, 12260, 12261, 12262, 12263, 12264, 12265, 12266, 12267, 12268, 12269, 12270, 12271, 12272, 12273, 12274, 12275, 12276, 12277, 12278, 12279, 12280, 12281, 12282, 12304, 12305, 12308, 12309, 12316, 12335, 12336, 12337, 12338, 12358, 12410, 12411, 12413, 12436, 12437, 12438, 12439, 12440, 12441, 12442, 12443, 12445, 12446, 12447, 12448, 12449, 12450, 12451, 12452, 12453, 12454, 12455, 12456, 12457, 12458, 12459, 12460, 12461, 12462, 12470, 12472, 12473, 12474, 12475, 12476, 12478, 12479, 12503, 12504, 12515, 12517, 12518, 12541, 12542, 12543, 12544, 12545, 12546, 12573, 12574, 12575, 12576, 12577, 12613, 12614, 12615, 12617, 12619, 12620, 12621, 12622, 12655, 12656, 
12658, 12661, 12671, 12672, 12673, 12674, 12675, 12676, 12678, 12682, 12690, 12691, 12692, 12693, 12694, 12695, 12696, 12697, 12698, 12699, 12700, 12701, 12702, 12703, 12704, 12705, 12706, 12707, 12708, 12709, 12710, 12711, 12712, 12713, 12714, 12719, 12720, 12721, 12722, 12723, 12724, 12725, 12726, 12727, 12728, 12729, 12730, 12731, 12733, 12726, 12737, 12739, 12740, 12742, 12743, 12744, 12745, 12747, 12748, 12749, 12750, 12751, 12752, 12753, 12754, 12755, 12756, 12757, 12758, 12759, 12764, 12762, 12763, 12764, 12765, 12766, 12767, 12768, 12769, 12770, 12771, 12772, 12773, 12774, 12775, 12776, 12777, 12778, 12779, 12780, 12781, 12782, 12784, 12785, 12795, 12796, 12797, 12798, 12799, 12800, 12801, 12802, 12803, 12804, 12805, 12806, 12807, 12808, 12809, 12810, 12811, 12812, 12813, 12814, 12815, 12820, 12821, 12822, 12830, 12831, 12832, 12833, 12834, 12835, 12836, 12837, 12839, 12840, 12841, 12842, 12843, 12844, 12845, 12846, 12847, 12848, 12849, 12850, 12851, 12852, 12853, 12854, 12855, 12856, 12858, 12859, 12860, 12861, 12862, 12863, 12864, 12865, 12866, 12867, 12868, 12869, 12871, 12872, 
12873, 12874, 12875, 12876, 12877, 12878, 12879, 12880, 12881, 12882, 12883, 12884, 12885, 12886, 12887, 12888, 12890, 12891, 12892, 12893, 12894, 12895, 12896, 12898, 12899, 12900, 12901, 12902, 12903, 12904, 12905, 12906, 12907, 12908, 12909, 12910, 12911, 12912, 12913, 12914, 12915, 12916, 12917, 12918, 12919, 12936, 12937, 12938, 12939, 12940, 12941, 12942, 12943, 12944, 12945, 12946, 12947, 12948, 12949, 12950, 12951, 12952, 12954, 12955, 12956, 12960, 12961, 12962, 12963, 12964, 12965, 12966, 12967, 12968, 12969, 12972, 12973, 12974, 12975, 12978, 12979, 12982, 12983, 12984, 12986, 12990, 12991, 12992, 12993, 12994, 12995, 12996, 12997, 12998, 12999, 13000, 13001, 13002, 13003, 13004, 13005, 13006, 13007, 13008, 13009, 13010, 13011, 13012, 13013, 13014, 13015, 13016, 13017, 13018, 13022, 13023, 13024, 13025, 13026, 13027, 13028, 13029, 13030, 13031, 13032, 13033, 13034, 13035, 13036, 13037, 13038, 13039, 13040, 13041, 13042, 13043, 13044, 13045, 13046, 13047, 13048, 13049, 13050, 13051, 13052, 13053, 13054, 13055, 13056, 13057, 13058, 13059, 13060, 13061, 13062, 13063, 13064, 13065, 
13066, 13067, 13068, 13069, 13070, 13071, 13072, 13073, 13074, 13075, 13076, 13077, 13078, 13079, 13080, 13081, 13082, 13083, 13085, 13087, 13088, 13089, 13090, 13091, 13092, 13093, 13094, 13095, 13096, 13097, 13098, 13099, 13100, 13101, 13102, 13103, 13105, 13106, 13107, 13108, 13109, 13110, 13111, 13112, 13113, 13114, 13115, 13116, 13117, 13118, 13119, 13120, 13121, 13122, 13132, 13133, 13134, 13135, 13136, 13137, 13138, 13139, 13140, 13141, 13142, 13143, 13144, 13145, 13146, 13147, 13148, 13149, 13150, 13151, 13152, 13153, 13154, 13155, 13156, 13157, 13158, 13159, 13161, 13162, 13165, 13166, 13169, 13170, 13171, 13172, 13173, 13174, 13175, 13176, 13177, 13178, 13179, 13180, 13181, 13182, 13183, 13184, 13185, 13186, 13187, 13188, 13189, 13190, 13191, 13192, 13193, 13194, 13199, 13200, 13201, 13202, 13203, 13204, 13205, 13206, 13207, 13208, 13209, 13210]
len(mod)

5436

In [15]:
# randomly pick one instrcution for each sample
web_pd.loc[:, "instruction"] = np.random.choice(instructions_base_long, len(web_pd))
web_pd.head(50)

,instruction,input,output,original_output
0,Think of yourself as efficient in deconstructi...,"The Aarhus is the airport of Aarhus, Denmark.","To extract triplets from the given text, we ne...","To extract triplets from the given text, we ne..."
1,Pretend you are highly skilled in breaking dow...,Aarhus airport serves the city of Aarhus.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
2,"Suppose you're an expert in text dissection, a...",Aarhus Airport is 25 metres above sea level.,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
3,Picture yourself as adept at understanding and...,Aarhus Airport is located in Tirstrup.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
4,"Suppose you're an expert in text dissection, a...",Aarhus Airport is operated by Aarhus Lufthavn ...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
5,Pretend you are highly skilled in breaking dow...,Aktieselskab is the operating organisation for...,"To extract triplets from the given text, we fi...","To extract triplets from the given text, we fi..."
6,"Suppose you're a maestro in textual analysis, ...",Aarhus Airport runway length is 2776.0.,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
7,Pretend that you are great at parsing a text a...,The runway length at Aarhus Airport is 2777.0 ...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
8,Envision yourself as skilled in analyzing text...,Aarhus Airport runway name is 10L/28R.,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
9,"Imagine you're an expert at deciphering text, ...","The runway name at Aarhus Airport is ""10R/28L"".","To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."


# Adding Triplets to Text Task

In [16]:
# instructions_triples_to_text = [
#     "Given the triplets below, write a short paragraph that satisfies the semantic meaning.",
#     "Using the provided triples below, write a short paragraph that extracts their meaning.",
#     "Taking the triples below as a guide, write a short paragraph that communicates their underlying meaning.",
#     "Utilizing the triples provided, generate a succinct paragraph that expresses their semantic value.",
#     "From the presented triple set, construct a paragraph that encapsulates their meaning.",
#     "Given these semantic triplets, craft a concise paragraph that interprets their meaning.",
#     "Given these semantic triplets, craft a concise paragraph that interprets their intended significance.",
#     "Considering the triplets below, write a short paragraph that communicates the meaning they suggest.",
#     "Adapt the given triples into a paragraph that represents their semantic meaning.",
#     "Convert the semantic triplets into a short paragraph that communicates their meaning.",
#     "Derive a short paragraph from the following triplets.",
#     "Interpret the following semantic triplets as text that captures their meaning.",
#     "Construct a short paragraph that captures the meaning of the following semantic triplets.",
#     "Turn the triplets below into a short textual representation that expresses their meaning.",
#     "Generate a short paragraph that satisfies the semantic meaning of the following triplets.",
#     "Process the following triplets into a concise paragraph that represents their meaning.",
#     "Reformat the triplets below into a paragraph that extracts their meaning.",
#     "Create a short paragraph that expresses the semantic meaning of the following triplets.",
#     "Transcribe the triplets into a short text that expresses their meaning.",
#     "Retrieve a short paragraph that communicates the underlying meaning of the following triplets.",
# ]

instructions_triples_to_text = [
    "Construct a short paragraph that captures the meaning of the following semantic triplets.",
    "Imagine you're proficient at deciphering a text, capably identifying all entities and their relationships, as well as forming a text from given triples. I will provide you with triples in the (subject | relationship | object) format. Your task is to scrutinize these triples and create text that encapsulates their semantic essence.",
    "Consider yourself an expert at text analysis, capable of accurately extracting all entities and their associations and then constructing text based on given triples. When supplied with triples in the (subject | relationship | object) format, it's your responsibility to evaluate these triples and craft text that conveys their semantic context.",
    "Assume you're skilled in interpreting text, efficiently identifying all entities and their interconnections, and also creating text from provided triples. I'll present you with triples in the format: (subject | relationship | object). Your assignment is to assess these triples and generate text that captures their semantic significance.",
    "Envision being adept at text dissection, able to pull out all entities and their ties, and then build text from given triples. When handed triples in the (subject | relationship | object) format, your job is to analyze these triples and produce text that embodies their semantic intent.",
    "Picture yourself as an expert in scrutinizing a text, effectively extracting all entities and their relationships and then constructing text based on the given triples. Once I supply you with triples in the (subject | relationship | object) format, your duty is to reexamine these triples and create text that imparts their semantic interpretation.",
    "Pretend you are skilled at deconstructing text, accurately discerning all entities and their correlations, and also formulating text from given triples. I'll give you triples in the form of (subject | relationship | object). Please inspect these triples and compose text that represents their semantic content.",
    "Suppose you're excellent at understanding and dissecting text, capable of identifying all entities and their connections and then generating text from such triples. When provided triples in the structure: (subject | relationship | object), your task is to review these triples and construct text that mirrors their semantic implications.",
    "Consider yourself a maestro in textual analysis, able to extract all entities and their interactions effectively, and can create text when supplied with triples. I will hand you triples in the (subject | relationship | object) format. Your mission is to vet these triples and craft text that portrays their semantic meaning.",
    "Think of yourself as proficient in text parsing, able to isolate all entities and their relationships, and then assemble text from provided triples. Upon receiving triples in the format: (subject | relationship | object), it's your responsibility to examine these triples and generate text that communicates their semantic context.",
    "Pretend you're an expert in textual dissection, with the capability to extract all entities and their ties, and then reassemble text when handed such triples. I'll give you triples in the format (subject | relationship | object). Your task is to peruse these triples and write text that expresses their semantic nuances."
]

In [17]:
triples_to_text = ttt_pd.rename(columns={"input":"output", "output":"input"})
triples_to_text = triples_to_text.drop(columns={"original_output"})
triples_to_text = triples_to_text.reindex(columns=order[:3])
triples_to_text

,instruction,input,output
0,"Suppose you're an expert in text dissection, a...","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","The Aarhus is the airport of Aarhus, Denmark."
1,Pretend you are highly skilled in breaking dow...,(Aarhus_Airport | cityServed | Aarhus),Aarhus airport serves the city of Aarhus.
2,"Imagine you're an expert at deciphering text, ...",(Aarhus_Airport | elevationAboveTheSeaLevel | ...,Aarhus Airport is 25 metres above sea level.
3,Pretend you are highly skilled in breaking dow...,(Aarhus_Airport | location | Tirstrup),Aarhus Airport is located in Tirstrup.
4,Think of yourself as efficient in deconstructi...,"(Aarhus_Airport | operatingOrganisation | ""Aar...",Aarhus Airport is operated by Aarhus Lufthavn ...
...,...,...,...
13206,"Imagine you're an expert at deciphering text, ...",(School of Business and Social Sciences at the...,In 1928 the School of Business and Social Scie...
13207,Picture yourself as adept at understanding and...,(School of Business and Social Sciences at the...,The Dean of the School of Business and Social ...
13208,"As someone proficient in text parsing, you sho...",(School of Business and Social Sciences at the...,The city of Aarhus in Denmark is the location ...
13209,"Suppose you're an expert in text dissection, a...",(Visvesvaraya_Technological_University | city ...,The campus address of the Acharya Institute of...


In [18]:
# randomly pick one instrcution for each sample
triples_to_text.loc[:, "instruction"] = np.random.choice(instructions_triples_to_text, len(triples_to_text))
triples_to_text.head(100)

,instruction,input,output
0,Pretend you are skilled at deconstructing text...,"(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","The Aarhus is the airport of Aarhus, Denmark."
1,"Assume you're skilled in interpreting text, ef...",(Aarhus_Airport | cityServed | Aarhus),Aarhus airport serves the city of Aarhus.
2,"Consider yourself an expert at text analysis, ...",(Aarhus_Airport | elevationAboveTheSeaLevel | ...,Aarhus Airport is 25 metres above sea level.
3,Suppose you're excellent at understanding and ...,(Aarhus_Airport | location | Tirstrup),Aarhus Airport is located in Tirstrup.
4,Imagine you're proficient at deciphering a tex...,"(Aarhus_Airport | operatingOrganisation | ""Aar...",Aarhus Airport is operated by Aarhus Lufthavn ...
...,...,...,...
95,Think of yourself as proficient in text parsin...,(Alpena_County_Regional_Airport | cityServed |...,Alpena County Regional Airport city served Alp...
96,"Consider yourself an expert at text analysis, ...",(Alpena_County_Regional_Airport | elevationAbo...,Alpena County Regional Airport elevation above...
97,Imagine you're proficient at deciphering a tex...,(Alpena_County_Regional_Airport | location | M...,Alpena County Regional Airport is found in Map...
98,"Consider yourself an expert at text analysis, ...",(Alpena_County_Regional_Airport | location | W...,Alpena County Regional Airport is found in Wil...


In [40]:
cnt = 0
for i in triples_to_text['instruction']:
    if i == "Construct a short paragraph that captures the meaning of the following semantic triplets.":
        cnt += 1
cnt

1149

In [19]:
from random import shuffle
# Shuffle the input triples where possible.
for i in range(len(triples_to_text['input'])):
    triple = triples_to_text['input'][i].split('\n')
    shuffle(triple)
    triples_to_text['input'][i] = "\n".join(map(str,triple))

In [20]:
triples_to_text.head(100)

,instruction,input,output
0,Pretend you are skilled at deconstructing text...,"(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","The Aarhus is the airport of Aarhus, Denmark."
1,"Assume you're skilled in interpreting text, ef...",(Aarhus_Airport | cityServed | Aarhus),Aarhus airport serves the city of Aarhus.
2,"Consider yourself an expert at text analysis, ...",(Aarhus_Airport | elevationAboveTheSeaLevel | ...,Aarhus Airport is 25 metres above sea level.
3,Suppose you're excellent at understanding and ...,(Aarhus_Airport | location | Tirstrup),Aarhus Airport is located in Tirstrup.
4,Imagine you're proficient at deciphering a tex...,"(Aarhus_Airport | operatingOrganisation | ""Aar...",Aarhus Airport is operated by Aarhus Lufthavn ...
...,...,...,...
95,Think of yourself as proficient in text parsin...,(Alpena_County_Regional_Airport | cityServed |...,Alpena County Regional Airport city served Alp...
96,"Consider yourself an expert at text analysis, ...",(Alpena_County_Regional_Airport | elevationAbo...,Alpena County Regional Airport elevation above...
97,Imagine you're proficient at deciphering a tex...,(Alpena_County_Regional_Airport | location | M...,Alpena County Regional Airport is found in Map...
98,"Consider yourself an expert at text analysis, ...",(Alpena_County_Regional_Airport | location | W...,Alpena County Regional Airport is found in Wil...


# Reflection on Triplets Task

In [21]:
reflection_data = load_dataset("UofA-LINGO/text-to-triples-train-eval-no-instructions")

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--text-to-triples-train-eval-no-instructions-33b1ae9cfb93b6f6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
# instructions_triples_reflection = [
#     "Fix the triplets generated from the text below.",
#     "Using the text below, fix the triples provided.",
#     "Correct the triplets that were generated from the text below.",
#     "Amend the triplets derived from the text below as necessary.",
#     "Correct any inaccuracies in the triplets extracted from the below text.",
#     "Review and adjust the triplets produced from the text below.",
#     "From the text given, rectify any errors present in the produced triplets.",
#     "Identify and correct any inaccuracies in the triplets generated from the provided text.",
#     "Examine the text below and make necessary corrections to the generated triplets.",
#     "Evaluate and correct the triples derived from the text below.",
#     "Make any necessary adjustments to the triples that have been extracted from the below text.",
#     "Verify and amend the triplets generated from the following text.",
#     "Using the provided text, rectify any mistakes in the generated triplets.",
#     "Analyze the generated triplets from the provided text and modify where required.",
#     "Using the text provided, review and improve the triplets as necessary.",
#     "Cross-check the triplets created from the following text and make corrections as needed.",
#     "Assess the text below and rectify the triplets as required.",
#     "Evaluate the triplets drawn from the following text and make any necessary adjustments.",
#     "Based on the text given, correct the generated triplets where needed.",
#     "From the text given, scrutinize and rectify the generated triplets as needed.",
# ]
instructions_triples_reflection = [
    "Identify and correct any inaccuracies in the triplets generated from the provided text.",
    "Imagine being proficient at deciphering a text, successfully drawing out all entities and their relationships. I will provide you with a text along with triples in the (subject | relationship | object) format. Your task is to scrutinize these triples in the context of the text and make any necessary corrections.",
    "Consider yourself as an expert at analyzing text, capable of accurately extracting all entities and their associations. When given a text and a set of triples formatted as (subject | relationship | object), it's your responsibility to examine these triples with reference to the text and rectify any errors.",
    "Think of yourself as adept at interpreting a piece of text, pulling out all entities and their interconnections with precision. I'll furnish you with a text and its associated triples in the structure of (subject | relationship | object). Your job is to review these triples considering the provided text and adjust any discrepancies.",
    "Picture yourself as being highly skilled in text dissection, with the ability to efficiently identify all entities and their ties. When provided a text along with triples in the (subject | relationship | object) format, you are to check these triples in light of the text and correct any inaccuracies.",
    "Assume you're an expert in scrutinizing a text and can effectively extract all its entities and their relationships. Once I supply you with a text and triples in the (subject | relationship | object) format, your assignment is to critically evaluate these triples against the text and amend any errors.",
    "Envision being highly proficient in parsing a text and being able to successfully discern all entities and their correlations. I'll give you a text and a set of triples in the form of (subject | relationship | object). Please verify these triples in relation to the text and fix any potential mistakes.",
    "Suppose you're excellent at understanding and dissecting a piece of text, capable of identifying all entities and their connections. When handed a text along with triples in the format: (subject | relationship | object), it's your duty to recheck these triples in the context of the text and correct any inaccuracies.",
    "Pretend you are skilled in breaking down text, with the ability to accurately pull out all entities and their interconnections. I'll hand you a text and triples in the structure of (subject | relationship | object). Your task is to reassess these triples against the text and rectify any issues.",
    "Consider yourself a maestro in textual analysis, able to extract all entities and their interactions effectively. I will supply you with a text and corresponding triples in the (subject | relationship | object) format. Please inspect these triples in light of the text and make the necessary adjustments.",
    "Think of yourself as proficient in deconstructing a text and extracting all entities and their ties. Once provided with a text and its related triples in the (subject | relationship | object) format, your mission is to cross-check these triples with the text and fix any inconsistencies."
]

In [23]:
reflection_pd = pd.DataFrame(reflection_data['train'])
# randomly pick one instrcution for each sample
reflection_pd.loc[:, "instruction"] = np.random.choice(instructions_triples_reflection, len(reflection_pd))
reflection_pd = reflection_pd.reindex(columns=order[:3])
reflection_pd.head(100)

,instruction,input,output
0,Identify and correct any inaccuracies in the t...,"Text: The Aarhus is the airport of Aarhus, Den...","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")"
1,Consider yourself a maestro in textual analysi...,Text: Aarhus airport serves the city of Aarhus...,(Aarhus_Airport | cityServed | Aarhus)
2,Consider yourself a maestro in textual analysi...,Text: Aarhus Airport is 25 metres above sea le...,(Aarhus_Airport | elevationAboveTheSeaLevel | ...
3,Imagine being proficient at deciphering a text...,Text: Aarhus Airport is located in Tirstrup.\n...,(Aarhus_Airport | location | Tirstrup)
4,Consider yourself a maestro in textual analysi...,Text: Aarhus Airport is operated by Aarhus Luf...,"(Aarhus_Airport | operatingOrganisation | ""Aar..."
...,...,...,...
95,Assume you're an expert in scrutinizing a text...,Text: Alpena County Regional Airport city serv...,(Alpena_County_Regional_Airport | cityServed |...
96,Assume you're an expert in scrutinizing a text...,Text: Alpena County Regional Airport elevation...,(Alpena_County_Regional_Airport | elevationAbo...
97,Identify and correct any inaccuracies in the t...,Text: Alpena County Regional Airport is found ...,(Alpena_County_Regional_Airport | location | M...
98,Envision being highly proficient in parsing a ...,Text: Alpena County Regional Airport is found ...,The generated triplets are already valid.


In [46]:
cnt = 0
for i in reflection_pd['instruction']:
    if i == "Identify and correct any inaccuracies in the triplets generated from the provided text.":
        cnt += 1
cnt

1086

In [47]:
print("Instruction:", reflection_pd['instruction'][1], '\n')
print(reflection_pd['input'][1], '\n')
print("Ground Truth:", reflection_pd['output'][1])

Instruction: Think of yourself as adept at interpreting a piece of text, pulling out all entities and their interconnections with precision. I'll furnish you with a text and its associated triples in the structure of (subject | relationship | object). Your job is to review these triples considering the provided text and adjust any discrepancies. 

Text: Aarhus airport serves the city of Aarhus.

Triples:
(Aarhus_Airport | cityServed | Aarhus)
(Aarhus_Airport | cityServed | Aarhus) 

Ground Truth: (Aarhus_Airport | cityServed | Aarhus)


In [29]:
import re

def split_ignore_quotes(input_string):
    pattern = r',(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^"]*"[^"]*(?:"[^"]*"[^"]*)*$)'  # Created with ChatGPT
    return re.split(pattern, input_string)

# Test cases
# print(split_ignore_quotes('(one, two)'))       # Output: ['(one', ' two)']
# print(split_ignore_quotes('"(one, two)"'))     # Output: ['"(one, two)"']
# print(split_ignore_quotes('a, b, "c, d", e')) # Output: ['a', ' b', ' "c, d"', ' e']
# print(split_ignore_quotes('(Aarhus_Airport, cityServed, "Aarhus, Denmark")')) # Output: ['(Arahus_Airport', ' cityServed', ' "Arahus, Denmark")']

# Combine Datasets

## Remove Modified Data From Explanations

In [31]:
exp_pd = web_pd.drop(index=mod)
exp_pd

,instruction,input,output,original_output
1,Picture yourself as adept at understanding and...,Aarhus airport serves the city of Aarhus.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
3,Think of yourself as efficient in deconstructi...,Aarhus Airport is located in Tirstrup.,"To extract triplets from the given text, you c...","To extract triplets from the given text, you c..."
6,Envision yourself as skilled in analyzing text...,Aarhus Airport runway length is 2776.0.,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
7,"Imagine you're an expert at deciphering text, ...",The runway length at Aarhus Airport is 2777.0 ...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
10,"Suppose you're an expert in text dissection, a...","Abilene, Texas is in the United States.","To extract triplets from the given text ""Abile...","To extract triplets from the given text ""Abile..."
...,...,...,...,...
13168,Think of yourself as efficient in deconstructi...,Switzerland's national anthem is the Swiss Psa...,The conversion process to extract triplets fro...,The conversion process to extract triplets fro...
13195,"Imagine you're an expert at deciphering text, ...",The Germans of Romania are one of the ethnic g...,"To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."
13196,"As someone proficient in text parsing, you sho...",1 Decembrie 1918 University in the state of Al...,"To extract triplets from the given text, the f...","To extract triplets from the given text, the f..."
13197,"Imagine you're an expert at deciphering text, ...","Romania, capital Bucharest, is home to the eth...","To extract triplets from the given text, we ca...","To extract triplets from the given text, we ca..."


## Combine Data

In [24]:
combined_few = pd.concat([ttt_pd, web_pd, triples_to_text, reflection_pd, few_pd], ignore_index=True)
combined_doc = pd.concat([ttt_pd, web_pd, triples_to_text, reflection_pd, doc_pd], ignore_index=True)
#combined = pd.concat([web_cl, exp_pd, triples_to_text], ignore_index=True)
combined_few

,instruction,input,output,original_output
0,"Suppose you're an expert in text dissection, a...","The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Pretend you are highly skilled in breaking dow...,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,"Imagine you're an expert at deciphering text, ...",Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Pretend you are highly skilled in breaking dow...,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Think of yourself as efficient in deconstructi...,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
...,...,...,...,...
97076,"Imagine you're an expert at deciphering text, ...","He won the first prize for piano, teaching, ch...",(rimsky-korsakov conservatory | located in the...,NaN
97077,Pretend that you are great at parsing a text a...,Tydavnet / Monaghan and Geel were officially t...,(tydavnet | located in the administrative terr...,NaN
97078,"Suppose you're an expert in text dissection, a...",Serbo-Croatian is a lingua franca in several o...,(serbia | located in the administrative territ...,NaN
97079,Pretend you are highly skilled in breaking dow...,A pivotal moment in UCSF history was the deal ...,(mission bay | located in the administrative t...,NaN


In [25]:
combined_doc

,instruction,input,output,original_output
0,"Suppose you're an expert in text dissection, a...","The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Pretend you are highly skilled in breaking dow...,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,"Imagine you're an expert at deciphering text, ...",Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Pretend you are highly skilled in breaking dow...,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Think of yourself as efficient in deconstructi...,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
...,...,...,...,...
55329,Picture yourself as adept at understanding and...,The United African National Council (UANC) was...,(ZANU - PF | country | Zimbabwean)\n(UANC Part...,NaN
55330,Think of yourself as efficient in deconstructi...,BBC Sessions is a live album by the British ro...,(I Feel Free – Ultimate Cream | performer | Cr...,NaN
55331,"As someone proficient in text parsing, you sho...",The Atheist Centre is an institution founded b...,(Saraswathi Gora | religion or worldview | Ath...,NaN
55332,"Imagine you're an expert at deciphering text, ...",The Battle of Naseby was a decisive engagement...,(Thomas Fairfax | military branch | Parliament...,NaN


In [49]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(combined)
    }
)

In [50]:
dataset.push_to_hub("UofA-LINGO/webnlg-rf-ske-combined", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/248 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(ske_pd),
        "valid": Dataset.from_pandas(ske_val)
    }
)

In [19]:
dataset.push_to_hub('UofA-LINGO/ske-train', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/195 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split valid to the Hub.
Resuming upload of the dataset shards.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(doc_pd)
    }
)
dataset.push_to_hub('UofA-LINGO/DocRED-train', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(combined_few)
    }
)
dataset.push_to_hub('UofA-LINGO/webnlg-rf-fewrel-combined', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(combined_doc)
    }
)
dataset.push_to_hub('UofA-LINGO/webnlg-rf-docred-combined', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
print(dataset['train'][5535])

{'instruction': "Picture yourself as adept at understanding and dissecting text, capable of efficiently identifying all entities and their interrelationships. Given a text, it's your task to isolate all potential triples, in the format of: (subject | relationship | object).", 'input': 'A Glastonbury Romance is the sequel to Wolf Solent and was written by John Cowper Powys.', 'output': '(A_Glastonbury_Romance | precededBy | Wolf_Solent)\n(A_Glastonbury_Romance | author | John_Cowper_Powys)', 'original_output': "['A_Glastonbury_Romance | precededBy | Wolf_Solent', 'A_Glastonbury_Romance | author | John_Cowper_Powys']"}


In [34]:
print(dataset['train'][13211])

{'instruction': "Assume you're highly competent in scrutinizing a piece of text and successfully distilling all its entities along with their connections. I'll provide a text, and you are to extract every possible triplet, following the convention: (subject | relationship | object). Detail the entire process systematically.", 'input': 'The Aarhus is the airport of Aarhus, Denmark.', 'output': 'To extract triplets from the given text, we need to identify the subject, predicate, and object. \n\nSubject: "Aarhus Airport"\nPredicate: "cityServed"\nObject: "Aarhus, Denmark"\n\nThe property "cityServed" is derived from the context of the sentence, where it implies that the airport serves the city of Aarhus.\n\nTherefore, here is the answer in the correct format:\n\n(Aarhus_Airport | cityServed | "Aarhus, Denmark")', 'original_output': 'To extract triplets from the given text, we need to identify the subject, predicate, and object. \n\nSubject: "Aarhus Airport"\nPredicate: "cityServed"\nObjec

In [265]:
print(dataset['train'][2452])

{'instruction': 'Turn the text below into triplets.', 'input': 'Adam Holloway was born in Faversham Kent, England.', 'output': '(Adam_Holloway, birthPlace, "Faversham, Kent, England")', 'original_output': '[\'Adam_Holloway | birthPlace | "Faversham, Kent, England"\']'}


In [266]:
print(dataset['train'][15661])

{'instruction': 'Convert the following text into semantic triplets give a stepwise explanation.', 'input': "Adam Holloway's Alma mater is Magdalene College, Cambridge.", 'output': 'To extract triplets from the given text, we follow these steps:\n\n1. Identify the subject: The subject in the given text is "Adam Holloway."\n\n2. Identify the property: The property in the given text is "alma mater."\n\n3. Identify the object: The object in the given text is "Magdalene College, Cambridge."\n\nTherefore, the triplet extracted from the given text is: \n"[\'Adam_Holloway | almaMater | Magdalene_College,_Cambridge\']"\n\nExplanation of properties:\n- "almaMater": It refers to the educational institution or university from which a person has graduated or attended.\n\nSource of the property in the text: \nThe property "alma mater" is a commonly used term to describe a person\'s educational background and is not specific to the given text.\n\nTherefore, here is the answer in the correct format:\n

In [21]:
# testing upload worked
data = load_dataset("UofA-LINGO/ske-train")
print(data)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/194728 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/21639 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--ske-train-096f4f55b7becb15/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'original_output'],
        num_rows: 194728
    })
    valid: Dataset({
        features: ['instruction', 'input', 'output', 'original_output'],
        num_rows: 21639
    })
})


In [22]:
tmp = pd.DataFrame(data['train'])
tmp[tmp.isna().any(axis=1)]

,instruction,input,output,original_output


In [6]:
# Check all triples were converted correctly in first part of data
ind = 0
for entity in data['train']['output'][:13211]:
    entity = entity.split('\n')
    for triple in entity:
        try:
            assert(triple[-1] == ')')
            #assert(len(triple.split(', ')) == 3)
        except:
            print("Short response")
            print(ind, triple)
    ind += 1

ind = 13212
# Check from explanation data next
for entity in data['train']['output'][13212:26422]:
    entity = entity.split("Therefore, here is the answer in the correct format:\n\n")[-1].split('\n')
    for triple in entity:
        try:
            assert(triple[-1] == ')')
            #assert(len(triple.split(', ')) == 3)
        except:
            print("Long response")
            print(ind, triple)
    ind += 1

ind = 38633
# Check for reflection data last
for entity in data['train']['output'][39633:]:
    if entity.find("The generated triplets are already valid.") != -1:
        ind += 1
        continue
    entity = entity.split('\n')
    for triple in entity:
        try:
            assert(triple[-1] == ')')
            #assert(len(triple.split(', ')) == 3)
        except:
            print("Reflection")
            print(ind, triple)
    ind += 1

In [17]:
#print(reflection_data['train']['input'][3547])
#print(reflection_data['train']['output'][3547])
print(reflection_data['train']['input'][0])
print(reflection_data['train']['output'][0])

The Aarhus is the airport of Aarhus, Denmark.

(Aarhus,_Denmark, is the airport of, Aarhus)
(Aarhus,_Denmark, is the airport of, Aarhus)
(Aarhus_Airport, cityServed, "Aarhus, Denmark")


# Update text_to_triples Test Prompts

In [5]:
ttt_train = pd.DataFrame(text_to_triples['train'])
ttt_train = ttt_train.rename(columns={"instruction":"instruction", "context":"input", "response":"original_output"})
#ttt_test = pd.DataFrame(text_to_triples['test'])
ttt_test = pd.DataFrame(webnlg_clean['test'])
ttt_test = ttt_test.rename(columns={"instruction":"instruction", "context":"input", "output":"original_output"})
ske_train = pd.DataFrame(ske['train'])
ske_test = pd.DataFrame(ske['test'])
ske_test = ske_test.rename(columns={"instruction":"instruction", "input":"input", "output":"original_output"})
docred_test = pd.DataFrame(load_dataset('UofA-LINGO/DocRED-dev-noins')['test'])
docred_test_webnlg = pd.DataFrame(load_dataset('UofA-LINGO/DocRED-dev-noins-webnlg-props')['test'])
fewrel_test = pd.DataFrame(load_dataset('UofA-LINGO/FewRel-val-noins')['test'])

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--DocRED-dev-noins-843db438000e4e10/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--DocRED-dev-noins-webnlg-props-0929650e24020a79/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/11200 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--FewRel-val-noins-ab77972e0177ef4e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
fewrel_test

,input,output
0,"In June 1987, the Missouri Highway and Transpo...",(cape girardeau bridge | crosses | mississippi...
1,Wilton Bridge was a major crossing of the ...,(wilton bridge | crosses | river wye)
2,State Road 66 begins at the eastern end of a t...,(toll bridge | crosses | wabash river)
3,"Albion Riverside, in London, is a high-end res...",(albert bridge | crosses | river thames)
4,"The Kurushima-Kaikyō Bridge, a toll road suspe...",(kurushima-kaikyō bridge | crosses | seto inla...
...,...,...
11195,"In 2013, Dunbar joined the CBS crime drama ""Th...",(the mentalist | main subject | fbi)
11196,"The project, however, took a different turn, a...",(the seduction of hillary rodham | main subjec...
11197,In June 2017 President of Catalonia Carles Pui...,(referendum | main subject | catalan independe...
11198,"This was in reaction to the Böhmermann affair,...",(böhmermann affair | main subject | jan böhmer...


In [40]:
ttt_test['input'][95]

'The birthdate of Brandon Carter is January 1, 1942.'

In [41]:
triples = []
cnt = 0
for out in ttt_train['original_output']:
    new_triples = ""
    out = re.split(r"\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split('|')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [42]:
triples = pd.DataFrame(triples, columns=['output'])
ttt_train.loc[:, "output"] = ttt_train.index.map(triples['output'])
ttt_train = ttt_train.reindex(columns=order)
ttt_train

,instruction,input,output,original_output
0,Generating triplets from the given text:,"The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Deriving triplets by analyzing the provided text:,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,Obtaining triples from the given text:,Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Deriving triplets by analyzing the provided text:,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Extracting triples by processing the given text:,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
...,...,...,...,...
13206,Creating triples by extracting information fro...,In 1928 the School of Business and Social Scie...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13207,Constructing triples by analyzing the given text:,The Dean of the School of Business and Social ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13208,Creating triples by extracting information fro...,The city of Aarhus in Denmark is the location ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13209,Generating triplets by parsing the specified t...,The campus address of the Acharya Institute of...,(Visvesvaraya_Technological_University | city ...,['Visvesvaraya_Technological_University | city...


In [43]:
ttt_test

,input,original_output
0,Turn Me On is a 35.1 minute long album produce...,"['Turn_Me_On_(album) | runtime | 35.1', 'Turn_..."
1,"The location of Trane is Swords, Dublin.","['Trane | location | Swords,_Dublin']"
2,"The Ciudad Ayala city, a part of Morelos with ...","['Ciudad_Ayala | populationMetro | 1777539', '..."
3,The 17068.8 millimeter long ALCO RS-3 has a di...,['ALCO_RS-3 | powerType | Diesel-electric_tran...
4,"Alan B. Miller Hall, in Virginia, USA, was des...",['Alan_B._Miller_Hall | architect | Robert_A._...
...,...,...
2150,Trane is a subsidiary company that was founded...,"['Trane | type | Subsidiary', 'Trane | foundin..."
2151,The long name of the Philippines is the Republ...,"['Philippines | longName | ""Republic of the Ph..."
2152,"Discovered by Grigory Neujmin, the asteroid 11...","['1147_Stavropolis | orbitalPeriod | 1249.6', ..."
2153,"The song Mermaid, which was written by Espen L...",['Mermaid_(song) | recordLabel | Columbia_Reco...


In [14]:
triples = []
cnt = 0
for out in ske_test['original_output']:
    new_triples = ""
    out = re.split(r"\', |\", ", out)  # Triples seem to be in multiple formats
    cnt += 1
    for triple in out:
        triple = triple.strip("[\'']").strip("\']'").split('|')
        try:
            assert(len(triple) == 3)
        except AssertionError:
            print(out)
            print(cnt)
        triple[0] = triple[0].strip(" \'")  # Need to do this one here to get rid of leading '
        try:
            new_triples += f'({triple[0].strip()} | {triple[1].strip()} | {triple[2].strip()})\n'
        except IndexError:
            print(out)
            print(cnt)
    new_triples = new_triples.strip()
    triples.append(new_triples)

In [16]:
triples = pd.DataFrame(triples, columns=['output'])
ske_test.loc[:, "output"] = ske_test.index.map(triples['output'])
ske_test = ske_test.reindex(columns=order)
ske_test

,instruction,input,output,original_output
0,extracting triples from following text,《不是所有时光都微笑》是2012年7月1日光明日报出版社出版的书籍，作者是蓝瞳,(不是所有时光都微笑 | 作者 | 蓝瞳)\n(不是所有时光都微笑 | 出版社 | 光明日报...,"['不是所有时光都微笑 | 作者 | 蓝瞳', '不是所有时光都微笑 | 出版社 | 光明日..."
1,extracting triples from following text,《鬼影实录2》是托德·威廉姆斯执导，布赖恩·波兰德主演的恐怖片,(鬼影实录2 | 导演 | 托德·威廉姆斯)\n(鬼影实录2 | 主演 | 布赖恩·波兰德),"['鬼影实录2 | 导演 | 托德·威廉姆斯', '鬼影实录2 | 主演 | 布赖恩·波兰德']"
2,extracting triples from following text,”这是明朝天启年间的首辅大学士叶向高为纪念尤溪籍靖边将领詹荣逝世七十年所写的《读史吊詹角山司马》诗,(詹荣 | 朝代 | 明朝)\n(叶向高 | 朝代 | 明朝),"['詹荣 | 朝代 | 明朝', '叶向高 | 朝代 | 明朝']"
3,extracting triples from following text,《在温柔中疼痛》是由网络作家宋江创作的一部网络文学作品，发表于2008年，正文语种为简体中文...,(在温柔中疼痛 | 作者 | 宋江),['在温柔中疼痛 | 作者 | 宋江']
4,extracting triples from following text,据22小编查询得知，1897年商务印书馆（TheCommercialPress（CP））在上...,(商务印书馆 | 成立日期 | 1897年),['商务印书馆 | 成立日期 | 1897年']
...,...,...,...,...
1145,extracting triples from following text,"公司介绍佛山晴杨铝业制品有限公司位于广东佛山市南海区高新科技产业开发区 , 公司专业开发、专...",(佛山晴杨铝业制品有限公司 | 总部地点 | 广东佛山市),['佛山晴杨铝业制品有限公司 | 总部地点 | 广东佛山市']
1146,extracting triples from following text,《一群土匪》是连载于起点中文网的小说，作者是玄斌天下,(一群土匪 | 作者 | 玄斌天下)\n(一群土匪 | 连载网站 | 起点中文网),"['一群土匪 | 作者 | 玄斌天下', '一群土匪 | 连载网站 | 起点中文网']"
1147,extracting triples from following text,《至尊兵王》是8难所著的都市小说，在创世中文网签约首发，2014年7月1日登上创世首页封面推荐,(至尊兵王 | 连载网站 | 创世中文网)\n(至尊兵王 | 作者 | 8难),"['至尊兵王 | 连载网站 | 创世中文网', '至尊兵王 | 作者 | 8难']"
1148,extracting triples from following text,《怪谈之有鬼》是连载于17k小说网的小说，作者是苍天邪笑,(怪谈之有鬼 | 作者 | 苍天邪笑)\n(怪谈之有鬼 | 连载网站 | 17k小说网),"['怪谈之有鬼 | 作者 | 苍天邪笑', '怪谈之有鬼 | 连载网站 | 17k小说网']"


In [21]:
for i in range(len(ske_test['original_output'])):
    original = re.split(r"\', |\", ", ske_test['original_output'][i])
    new = ske_test['output'][i].split('\n')
    assert(len(original) == len(new))

In [22]:
# randomly pick one instrcution for each sample
#ttt_train.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(ttt_train))
#ttt_train.head(50)
ske_test.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(ske_test))
ske_test.head(50)

,instruction,input,output,original_output
0,"As someone proficient in text parsing, you sho...",《不是所有时光都微笑》是2012年7月1日光明日报出版社出版的书籍，作者是蓝瞳,(不是所有时光都微笑 | 作者 | 蓝瞳)\n(不是所有时光都微笑 | 出版社 | 光明日报...,"['不是所有时光都微笑 | 作者 | 蓝瞳', '不是所有时光都微笑 | 出版社 | 光明日..."
1,"As someone proficient in text parsing, you sho...",《鬼影实录2》是托德·威廉姆斯执导，布赖恩·波兰德主演的恐怖片,(鬼影实录2 | 导演 | 托德·威廉姆斯)\n(鬼影实录2 | 主演 | 布赖恩·波兰德),"['鬼影实录2 | 导演 | 托德·威廉姆斯', '鬼影实录2 | 主演 | 布赖恩·波兰德']"
2,Envision yourself as skilled in analyzing text...,”这是明朝天启年间的首辅大学士叶向高为纪念尤溪籍靖边将领詹荣逝世七十年所写的《读史吊詹角山司马》诗,(詹荣 | 朝代 | 明朝)\n(叶向高 | 朝代 | 明朝),"['詹荣 | 朝代 | 明朝', '叶向高 | 朝代 | 明朝']"
3,Picture yourself as adept at understanding and...,《在温柔中疼痛》是由网络作家宋江创作的一部网络文学作品，发表于2008年，正文语种为简体中文...,(在温柔中疼痛 | 作者 | 宋江),['在温柔中疼痛 | 作者 | 宋江']
4,Convert the following text into semantic tripl...,据22小编查询得知，1897年商务印书馆（TheCommercialPress（CP））在上...,(商务印书馆 | 成立日期 | 1897年),['商务印书馆 | 成立日期 | 1897年']
5,Pretend that you are great at parsing a text a...,刘惠贤 1932年10月出生，笔名学谦，女，中共党员，陕西省高陵县人，毕业于新疆大学，曾任新...,(刘惠贤 | 出生地 | 陕西省高陵县)\n(刘惠贤 | 毕业院校 | 新疆大学)\n(刘惠...,"['刘惠贤 | 出生地 | 陕西省高陵县', '刘惠贤 | 毕业院校 | 新疆大学', '刘..."
6,Envision yourself as skilled in analyzing text...,《王任重文集》是1999年12月中央文献出版社出版的图书，作者是王任重,(王任重文集 | 作者 | 王任重)\n(王任重文集 | 出版社 | 中央文献出版社),"['王任重文集 | 作者 | 王任重', '王任重文集 | 出版社 | 中央文献出版社']"
7,Envision yourself as skilled in analyzing text...,不老男神主演的《放羊的星星》相信大家都看过，电视剧主题曲《我们的纪念》也同样很入耳，李雅微的演唱,(我们的纪念 | 歌手 | 李雅微)\n(放羊的星星 | 主演 | 不老男神),"['我们的纪念 | 歌手 | 李雅微', '放羊的星星 | 主演 | 不老男神']"
8,Envision yourself as skilled in analyzing text...,《靠自己去成功》是长江文艺出版社出版的图书，作者是刘墉,(靠自己去成功 | 作者 | 刘墉)\n(靠自己去成功 | 出版社 | 长江文艺出版社),"['靠自己去成功 | 作者 | 刘墉', '靠自己去成功 | 出版社 | 长江文艺出版社']"
9,Assume you're highly competent in scrutinizing...,《唯一继承者》是中国台湾省无线卫星电视台出品的爱情喜剧，由陈东汉执导，张睿家、宋芸桦、杨晴瑄...,(唯一继承者 | 主演 | 宋芸桦)\n(唯一继承者 | 主演 | 张睿家)\n(唯一继承者...,"['唯一继承者 | 主演 | 宋芸桦', '唯一继承者 | 主演 | 张睿家', '唯一继承..."


In [25]:
dataset = DatasetDict(
    {
        "test": Dataset.from_pandas(ske_test)
    }
)

In [26]:
dataset.push_to_hub("UofA-LINGO/ske-test", private=True)

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
fewrel_test.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(fewrel_test))

In [8]:
fewrel_test = fewrel_test.reindex(columns=['instruction', 'input', 'output'])
fewrel_test

,instruction,input,output
0,Pretend that you are great at parsing a text a...,"In June 1987, the Missouri Highway and Transpo...",(cape girardeau bridge | crosses | mississippi...
1,Pretend you are highly skilled in breaking dow...,Wilton Bridge was a major crossing of the ...,(wilton bridge | crosses | river wye)
2,Pretend that you are great at parsing a text a...,State Road 66 begins at the eastern end of a t...,(toll bridge | crosses | wabash river)
3,Envision yourself as skilled in analyzing text...,"Albion Riverside, in London, is a high-end res...",(albert bridge | crosses | river thames)
4,"Imagine you're an expert at deciphering text, ...","The Kurushima-Kaikyō Bridge, a toll road suspe...",(kurushima-kaikyō bridge | crosses | seto inla...
...,...,...,...
11195,"Suppose you're an expert in text dissection, a...","In 2013, Dunbar joined the CBS crime drama ""Th...",(the mentalist | main subject | fbi)
11196,Pretend that you are great at parsing a text a...,"The project, however, took a different turn, a...",(the seduction of hillary rodham | main subjec...
11197,Convert the following text into semantic tripl...,In June 2017 President of Catalonia Carles Pui...,(referendum | main subject | catalan independe...
11198,Envision yourself as skilled in analyzing text...,"This was in reaction to the Böhmermann affair,...",(böhmermann affair | main subject | jan böhmer...


In [10]:
dataset = DatasetDict(
    {
        "test": Dataset.from_pandas(fewrel_test)
    }
)
dataset.push_to_hub("UofA-LINGO/FewRel-val-test", private=True)

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
test_dat = pd.DataFrame(load_dataset("UofA-LINGO/webnlg-test-cleaned")['test'])

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--webnlg-test-cleaned-9509f08c0f4d5e03/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
test_dat.head() # 743, 963, 1149, 1212, 1673

,instruction,input,output,original_output
0,Think of yourself as efficient in deconstructi...,Turn Me On is a 35.1 minute long album produce...,(Turn_Me_On_(album) | runtime | 35.1)\n(Turn_M...,"['Turn_Me_On_(album) | runtime | 35.1', 'Turn_..."
1,Assume you're highly competent in scrutinizing...,"The location of Trane is Swords, Dublin.","(Trane | location | Swords,_Dublin)","['Trane | location | Swords,_Dublin']"
2,Pretend that you are great at parsing a text a...,"The Ciudad Ayala city, a part of Morelos with ...",(Ciudad_Ayala | populationMetro | 1777539)\n(C...,"['Ciudad_Ayala | populationMetro | 1777539', '..."
3,Picture yourself as adept at understanding and...,The 17068.8 millimeter long ALCO RS-3 has a di...,(ALCO_RS-3 | powerType | Diesel-electric_trans...,['ALCO_RS-3 | powerType | Diesel-electric_tran...
4,Envision yourself as skilled in analyzing text...,"Alan B. Miller Hall, in Virginia, USA, was des...",(Alan_B._Miller_Hall | architect | Robert_A._M...,['Alan_B._Miller_Hall | architect | Robert_A._...


In [11]:
ind = 1673
print(test_dat['instruction'][ind])
print(test_dat['input'][ind])

Suppose you're an expert in text dissection, able to accurately pull out all entities and their ties from any text. I will provide you with a text, and your task is to excavate all conceivable triples, following the template: (subject | relationship | object).
The album Take It Off was preceded by the 35 minute 10 second album Turn Me On.


# Making NYT Dataset

In [9]:
training = pd.read_csv("C:/Users/tyms4/OneDrive/Documents/Work/UofAResearch/lingo/data/nyt21/new_train.csv")
validation = pd.read_csv("C:/Users/tyms4/OneDrive/Documents/Work/UofAResearch/lingo/data/nyt21/new_valid.csv")
test = pd.read_csv("C:/Users/tyms4/OneDrive/Documents/Work/UofAResearch/lingo/data/nyt21/new_test.csv")

In [19]:
# Renaming columns to follow our instruction, input, output scheme.
training = training.rename(columns={"context(string)":"input", "response(string)":"output", "instruction(string)":"instruction"})
validation = validation.rename(columns={"context(string)":"input", "response(string)":"output", "instruction(string)":"instruction"})
test = test.rename(columns={"context(string)":"input", "response(string)":"output", "instruction(string)":"instruction"})

# Reorder
training = training.reindex(columns=order[:3])
validation = validation.reindex(columns=order[:3])
test = test.reindex(columns=order[:3])

In [23]:
# Give new instructions, randomly pick one instrcution for each sample
training.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(training))
training

,instruction,input,output
0,Transcribe the following text into semantic tr...,"Metric brands include Honda , Kawasaki , Yama...",['Japan | /location/location/contains | Kawasa...
1,Generate semantic triplets from the text beneath.,Stony Brook has Suffolk County 's only trauma...,['Suffolk County | /location/location/contains...
2,Obtain triplets from the text that follows.,His father retired as a professor of Spanish ...,['Lawrence | /location/location/contains | Uni...
3,Obtain triplets from the text that follows.,Spain said Andalusia had not previously appoi...,['Andalusia | /location/administrative_divisio...
4,Transcribe the following text into semantic tr...,"Mr . Lack , who had most recently been presid...",['Howard Stringer | /business/person/company |...
...,...,...,...
62330,Generate semantic triplets from the text beneath.,"Ms . May 's debut film , '' A NEW LEAF '' ( 1...",['Beacon Hill | /location/neighborhood/neighbo...
62331,Transpose the text underneath into triplets.,He died with six other soldiers on a rescue m...,['Iraq | /location/location/contains | Sadr Ci...
62332,Pull out triplets from the text underneath.,The package gave Mississippi about five times...,['Haley Barbour | /people/person/place_lived |...
62333,Turn the text below into triplets.,"At one precinct in Sadr City , the giant slum...",['Baghdad | /location/location/contains | Sadr...


In [24]:
# Give new instructions, randomly pick one instrcution for each sample
validation.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(validation))
validation

,instruction,input,output
0,Pull out triplets from the text underneath.,Homeowners in parts of Palm Beach County in F...,['Florida | /location/location/contains | Palm...
1,Construct semantic triplets from the text prov...,"In Spain , protests unfolded in nine cities ,...",['Barcelona | /location/administrative_divisio...
2,Draw out semantic triplets from the text provi...,A bank loan officer from Dix Hills has been a...,['Nassau County | /location/location/contains ...
3,Transcribe the following text into semantic tr...,"Europe and Russia are next in line , followed...",['Europe | /location/location/contains | Russia']
4,Process the text below into triplets.,The pope retains eight other titles : bishop ...,['Italy | /location/country/capital | Rome']
...,...,...,...
308,Draw out semantic triplets from the text provi...,If Bashar Assad were watching you guys booing...,['Bashar al-Assad | /people/person/nationality...
309,Bring out triplets from the text that's below.,Before he entered a state insane asylum in 19...,['Louis Armstrong | /people/person/place_of_bi...
310,Process the text below into triplets.,"In lieu of flowers , donations may be sent to...",['Boston | /location/location/contains | Massa...
311,Process the text below into triplets.,In one of the most fiercely contested races i...,['John Hostettler | /people/person/place_lived...


In [25]:
# Give new instructions, randomly pick one instrcution for each sample
test.loc[:, "instruction"] = np.random.choice(instructions_base_short, len(test))
test

,instruction,input,output
0,Decipher the text underneath into triplets.,Gov . Tim Pawlenty of Minnesota ordered the s...,['Tim Pawlenty | /people/person/place_lived | ...
1,Turn the text below into triplets.,The aid group Doctors Without Borders said th...,['Guinea | /location/country/capital | Conakry...
2,Convert the following text into semantic tripl...,The American people can see what is happening...,['Ben Nelson | /people/person/place_lived | Ne...
3,Transform the subsequent text into semantic tr...,"For millions , it was a tough day of coping o...",['Westchester | /location/location/contains | ...
4,Convert the following text into semantic tripl...,The United States maintains diplomatic relati...,['Damascus | /location/administrative_division...
...,...,...,...
364,Bring out triplets from the text that's below.,Mr . Bernanke 's tone of urgency echoed the c...,['Kent Conrad | /people/person/place_lived | N...
365,Convert the following text into semantic tripl...,THE WAY WE LIVE NOW : 3-18-07 Kwame Anthony A...,['Kwame Anthony Appiah | /business/person/comp...
366,Transform the subsequent text into semantic tr...,"DREXLER Beatrice C . , who ran Candy Mountain...",['Rockland County | /location/location/contain...
367,Bring out triplets from the text that's below.,"In another letter on Tuesday , two Democratic...",['Sheldon Whitehouse | /people/person/place_li...


In [65]:
print(training['input'][1430])
print(training['output'][1430])
print()
print(training['input'][2505])
print(training['output'][2505])

 Britain returned Hong Kong to China in 1997 , but Hong Kong remains a separate customs area from the mainland and is a separate member of the World Trade Organization . 
['Hong Kong | /location/administrative_division/country | China', 'China | /location/country/administrative_divisions | Hong Kong']

 The gold medalist , Dale Begg-Smith of Australia , is a true mogul 's mogul . 
['Dale Begg-Smith | /people/person/nationality | Australia']


In [44]:
cnt = 0
ind = 0
l = []
for entry in training['output']:
    if len(entry.split("\',")) > 1:
        cnt += 1
        l.append((ind,entry))
    ind += 1

In [45]:
cnt

9535

In [49]:
l[4264]

(27955,
 "['Marty Markowitz | /people/person/place_of_birth | Brooklyn', 'Marty Markowitz | /people/person/place_lived | Brooklyn']")

In [53]:
for item in l:
    print(item)

(3, "['Andalusia | /location/administrative_division/country | Spain', 'Spain | /location/country/administrative_divisions | Andalusia']")
(15, "['Shanghai | /location/administrative_division/country | China', 'China | /location/country/administrative_divisions | Shanghai']")
(23, "['Montreal | /location/administrative_division/country | Canada', 'Canada | /location/country/administrative_divisions | Montreal']")
(25, "['Ehud Olmert | /people/person/nationality | Israel', 'Iran | /location/country/capital | Tehran']")
(26, "['Camiguin | /location/administrative_division/country | Philippines', 'Philippines | /location/country/administrative_divisions | Camiguin']")
(27, "['Dubai | /location/administrative_division/country | United Arab Emirates', 'United Arab Emirates | /location/country/administrative_divisions | Dubai']")
(38, "['Michael Bloomberg | /people/person/place_lived | New York City', 'George Pataki | /people/person/place_lived | New York']")
(47, "['Ali al-Naimi | /business

In [3]:
d = load_dataset('UofA-LINGO/webnlg-combined')['train']

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--webnlg-combined-c8980a098143d7ae/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
d

Dataset({
    features: ['instruction', 'input', 'output', 'original_output'],
    num_rows: 39633
})

In [8]:
d[30000]

{'instruction': 'Given these semantic triplets, craft a concise paragraph that interprets their intended significance.',
 'input': '(Andra_(singer), genre, Rhythm_and_blues)\n(Andra_(singer), activeYearsStartYear, 2000)',
 'output': 'Andra is a rhythm and blues singer who became active in the year 2000.',
 'original_output': None}

In [7]:
ttt_pd

,instruction,input,output,original_output
0,Generating triplets from the given text:,"The Aarhus is the airport of Aarhus, Denmark.","(Aarhus_Airport | cityServed | ""Aarhus, Denmark"")","['Aarhus_Airport | cityServed | ""Aarhus, Denma..."
1,Deriving triplets by analyzing the provided text:,Aarhus airport serves the city of Aarhus.,(Aarhus_Airport | cityServed | Aarhus),['Aarhus_Airport | cityServed | Aarhus']
2,Obtaining triples from the given text:,Aarhus Airport is 25 metres above sea level.,(Aarhus_Airport | elevationAboveTheSeaLevel | ...,['Aarhus_Airport | elevationAboveTheSeaLevel |...
3,Deriving triplets by analyzing the provided text:,Aarhus Airport is located in Tirstrup.,(Aarhus_Airport | location | Tirstrup),['Aarhus_Airport | location | Tirstrup']
4,Extracting triples by processing the given text:,Aarhus Airport is operated by Aarhus Lufthavn ...,"(Aarhus_Airport | operatingOrganisation | ""Aar...","['Aarhus_Airport | operatingOrganisation | ""Aa..."
...,...,...,...,...
13206,Creating triples by extracting information fro...,In 1928 the School of Business and Social Scie...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13207,Constructing triples by analyzing the given text:,The Dean of the School of Business and Social ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13208,Creating triples by extracting information fro...,The city of Aarhus in Denmark is the location ...,(School of Business and Social Sciences at the...,['School of Business and Social Sciences at th...
13209,Generating triplets by parsing the specified t...,The campus address of the Acharya Institute of...,(Visvesvaraya_Technological_University | city ...,['Visvesvaraya_Technological_University | city...


In [8]:
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(ttt_pd[0:100])
    }
)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'original_output'],
        num_rows: 100
    })
})

In [10]:
dataset.push_to_hub("UofA-LINGO/webnlg-mini", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]